#### What Require Inputs in this Notebook
1. [Fill in File Input](#Load-in-Data)
2. [Right or Left Hand](#Right-or-Left-Hand)
3. [Output ShotCSV](#Output-ShotCSV)
4. [Change to atNet (by Player) from tagger later](#At-Net)
5. [EDA and get Stats Here](#EDA)
6. [Output PointCSV for Visuals](#PRINT-POINTCSV-VISUALS)
7. [Output PointCSV for Upload](#Print-POINTCSV-UPLOAD)

# Table of Contents
1. [Load in Data](#Load-in-Data)
- Error Checking
- Add Columns
- [Output ShotCSV](#Output-ShotCSV)
2. [Create Point DF](#Create-Point-DF)
- Add Columns
- [Output PointCSV for Visuals](#PRINT-POINTCSV-VISUALS)
- [Output PointCSV for Upload](#Print-POINTCSV-UPLOAD)    
        - cut out points with no timestamp position    
        - atNetPlayer1 has values of the player name for display purposes instead of boolean values

# [Click for Summary Stats and EDA HERE](#EDA)

## CJ Requests
- add cell in the beggining of the notebook to add metat data here (Date	Division	Event	lineupPosition	matchDetails	matchVenue	opponentTeam	player1Name	player2Name	player1Hand	player2Hand	Round	Surface)
- update the fill setscore to fill after the first iteration (ex. everytime the pointscore and gamescore is 0-0); add this after fixing the pointscore and gamescores, after error check 20
- add cell to error check if the server names are alternating correctly for the tiebreaker scores and gameScores

# TODO

###  Cameryn 
[Depth Count for each Point](#Add-Columns:-Depths-Count-(Short,-Deep))

[EDA for Depth of GroundStrokes](#EDA)

Add...
- Depth Count Average for each point for each player
- Do this each for all types of shots (Fh, Bh, Slice, Volley)

### Angelo 
[Change atNetPlayer to be aggregated from Coordinate Data](#Add-Column:-atNetPlayer1,-atNetPlayer2)
- Check what the differences are from the manually tagged data and coordinate data
- May want to see if shotLocation and shotContact are close to each other to determine if it's a volley

### Alexis 
[Change isApproach to be aggregatead from Coordinate Data](####-Add-Column:-isAprroach)
- see if the next shotInRally coordinates are further  up into the court and if the next shot is a volley?


# Notebook Start

### Load in Data

In [7]:
import pandas as pd
import numpy as np

In [8]:
# Put your Uncleaned .csv file name here

# your_file_name = "RudyQuan_YutaShimizu_points.csv"
# your_file_name ='Cleaned [WEBSITE] Elise_Wagle_Texas_Tech_Full_Match - Cleaned [WEBSITE] Elise_Wagle_Texas_Tech_Full_Match.csv'
# shot_data = pd.read_csv(your_file_name)

shot_data = pd.read_csv('cleaned_shot_data.csv')

In [9]:
# Check existing columns
shot_data.columns

Index(['pointScore', 'gameScore', 'setScore', 'isPointStart', 'pointStartTime',
       'isPointEnd', 'pointEndTime', 'pointNumber', 'isBreakPoint',
       'shotInRally', 'side', 'serverName', 'serverFarNear', 'firstServeIn',
       'firstServeZone', 'firstServeXCoord', 'firstServeYCoord',
       'secondServeIn', 'secondServeZone', 'secondServeXCoord',
       'secondServeYCoord', 'isAce', 'shotContactX', 'shotContactY',
       'shotDirection', 'shotFhBh', 'isSlice', 'isVolley', 'isOverhead',
       'isApproach', 'isDropshot', 'isExcitingPoint', 'atNetPlayer1',
       'atNetPlayer2', 'isLob', 'shotLocationX', 'shotLocationY', 'isWinner',
       'isErrorWideR', 'isErrorWideL', 'isErrorNet', 'isErrorLong',
       'clientTeam', 'Date', 'Division', 'Event', 'lineupPosition',
       'matchDetails', 'matchVenue', 'opponentTeam', 'player1Name',
       'player2Name', 'player1Hand', 'player2Hand', 'Round', 'Surface',
       'Notes'],
      dtype='object')

In [10]:
desired_order = ['pointScore', 'gameScore', 'setScore', 'isPointStart', 'pointStartTime',
       'isPointEnd', 'pointEndTime', 'pointNumber', 'isBreakPoint',
       'shotInRally', 'side', 'serverName', 'serverFarNear', 'firstServeIn',
       'firstServeZone', 'firstServeXCoord', 'firstServeYCoord',
       'secondServeIn', 'secondServeZone', 'secondServeXCoord',
       'secondServeYCoord', 'isAce', 'shotContactX', 'shotContactY',
       'shotDirection', 'shotFhBh', 'isSlice', 'isVolley', 'isOverhead',
       'isApproach', 'isDropshot', 'isExcitingPoint', 'atNetPlayer1',
       'atNetPlayer2', 'isLob', 'shotLocationX', 'shotLocationY', 'isWinner',
       'isErrorWideR', 'isErrorWideL', 'isErrorNet', 'isErrorLong',
       'clientTeam', 'Date', 'Division', 'Event', 'lineupPosition',
       'matchDetails', 'matchVenue', 'opponentTeam', 'player1Name',
       'player2Name', 'player1Hand', 'player2Hand', 'Round', 'Surface',
       'Notes']

shot_data = shot_data.reindex(columns=desired_order)

In [11]:
# shot_data[['clientTeam', 'Date', 'Division', 'Event', 'lineupPosition',
#        'matchDetails', 'matchVenue', 'opponentTeam', 'player1Name',
#        'player2Name', 'player1Hand', 'player2Hand', 'Round', 'Surface',
#        'Notes']]

# shot_data['clientTeam'] = 'UCLA'
# shot_data['Date'] = ''
# shot_data['Division'] = ''
# shot_data['Event'] = ''
# shot_data['lineupPosition'] = ''
# shot_data['matchDetails'] = ''
# shot_data['matchVenue'] = ''
# shot_data['opponentTeam'] = ''
# shot_data['player1Name'] = ''
# shot_data['player2Name'] = ''
# shot_data['player1Hand'] = ''
# shot_data['player2Hand'] = ''
# shot_data['Round'] = ''
# shot_data['Surface'] = ''
# shot_data['Notes'] = ''

## ASSIGN Player Names (CJ)

In [13]:
player1_new_name = shot_data['player1Name'].iloc[0]
player2_new_name = shot_data['player2Name'].iloc[0]

# Replace values in the serverName column
shot_data['serverName'] = shot_data['serverName'].replace({'Player1': player1_new_name, 'Player2': player2_new_name})

In [14]:
shot_data['serverName']

0      Motka Komac
1      Motka Komac
2      Motka Komac
3      Motka Komac
4      Motka Komac
          ...     
680    Elise Wagle
681    Elise Wagle
682    Elise Wagle
683    Elise Wagle
684    Elise Wagle
Name: serverName, Length: 685, dtype: object

<!-- ### Error Check: 
#### If data includes firstServeZone but no "1" for firstServeIn, same for secondServeZone/secondServeIn -->

## Error Check 1:
#### If returnData exists, separate values of "Forehand/Backhand Crosscourt/Down the Line" into shotFhBh and shotDirection

In [17]:
# Check if 'returnData' column exists
if 'returnData' in shot_data.columns:
    # Replace NaN values in returnData column with an empty string
    shot_data['returnData'].fillna('', inplace=True)

    # Check if "backhand" or "forehand" is in returnData and update shotFhBh accordingly
    shot_data.loc[shot_data['returnData'].str.contains('backhand', case=False), 'shotFhBh'] = 'Backhand'
    shot_data.loc[shot_data['returnData'].str.contains('forehand', case=False), 'shotFhBh'] = 'Forehand'

    # Check if "Crosscourt" or "Down the Line" is in returnData and update shotDirection accordingly
    shot_data.loc[shot_data['returnData'].str.contains('Crosscourt', case=False), 'shotDirection'] = 'Crosscourt'
    shot_data.loc[shot_data['returnData'].str.contains('Down the Line', case=False), 'shotDirection'] = 'Down the Line'

    # Display the modified shot_data
    print(shot_data[['returnData', 'shotDirection', 'shotFhBh']].head(10))
else:
    print("Check Passed: Column 'returnData' does not exist.")


Check Passed: Column 'returnData' does not exist.


## Error Check 2:
#### Change ShotType column to shotFhBh column if using old tagger

In [19]:
if 'shotType' in shot_data.columns:
    # Rename the column
    shot_data.rename(columns={'shotType': 'shotFhBh'}, inplace=True)
    print("Column renamed successfully.")
else:
    print("Check Passed: Column 'shotType' does not exist.")

Check Passed: Column 'shotType' does not exist.


## Error Check 3:
#### From Old Tagger, check that shotFhBh doesn't include Direction and Fh/Bh

In [21]:
shot_data.loc[shot_data['shotFhBh'] == 'Forehand Crosscourt', 'shotDirection'] = 'Crosscourt'
shot_data.loc[shot_data['shotFhBh'] == 'Forehand Crosscourt', 'shotFhBh'] = 'Forehand'

shot_data.loc[shot_data['shotFhBh'] == 'Backhand Crosscourt', 'shotDirection'] = 'Crosscourt'
shot_data.loc[shot_data['shotFhBh'] == 'Backhand Crosscourt', 'shotFhBh'] = 'Backhand'

shot_data.loc[shot_data['shotFhBh'] == 'Forehand Down the Line', 'shotDirection'] = 'Down the Line'
shot_data.loc[shot_data['shotFhBh'] == 'Forehand Down the Line', 'shotFhBh'] = 'Forehand'

shot_data.loc[shot_data['shotFhBh'] == 'Backhand Down the Line', 'shotDirection'] = 'Down the Line'
shot_data.loc[shot_data['shotFhBh'] == 'Backhand Down the Line', 'shotFhBh'] = 'Backhand'

print(shot_data[['shotDirection', 'shotFhBh']].head(10))

   shotDirection  shotFhBh
0            NaN       NaN
1  Down the Line  Forehand
2  Down the Line  Backhand
3     Crosscourt  Forehand
4     Crosscourt  Forehand
5  Down the Line  Backhand
6     Crosscourt  Forehand
7     Crosscourt  Forehand
8     Crosscourt  Forehand
9     Crosscourt  Backhand


## Error Check 4:
#### Check that Player1Name and Player2Name have values

In [23]:
fillInPlayer1Name = "Choose_Player1Name_Here"
fillInPlayer2Name = "Choose_Player2Name_Here"

if shot_data.at[0, 'player1Name'] == None:
    print("player1Name was blank. Filling with 'fillInplayer1Name'")
    shot_data.at[0, 'player1Name'] = fillInPlayer1Name
else:
    print("Check Passed: player1Name is not blank. Current value:", shot_data.at[0, 'player1Name'])

# Check if player2Name is blank, if so, fill it with 'fillInplayer2Name'
if shot_data.at[0, 'player2Name'] == None:
    print("player2Name was blank. Filling with 'fillInplayer2Name'")
    shot_data.at[0, 'player2Name'] = fillInPlayer2Name
else:
    print("Check Passed: player2Name is not blank. Current value:", shot_data.at[0, 'player2Name'])


Check Passed: player1Name is not blank. Current value: Elise Wagle
Check Passed: player2Name is not blank. Current value: Motka Komac


## Error Check 5:
#### Check if there are missing 'Deuce' or 'Ad' sides in side column

In [25]:
# Filter out rows with 'side' values other than 'Deuce' and 'Ad'
filtered_data = shot_data[shot_data['side'].isin(['Deuce', 'Ad'])]

# Rows that don't meet the condition
rows_not_meeting_condition = shot_data[~shot_data['side'].isin(['Deuce', 'Ad'])]

# Count of rows not meeting the condition
count_rows_not_meeting_condition = len(rows_not_meeting_condition)

print("Rows with Side that aren't Deuce or Ad:", count_rows_not_meeting_condition)

# Output rows if you want to see which rows
# print("Rows not meeting the condition:")
# print(rows_not_meeting_condition)


Rows with Side that aren't Deuce or Ad: 0


## Error Check 6: 
#### Fills in PointNumber if not there

In [27]:
 # Check if both conditions passed
if 'pointNumber' in shot_data.columns and not shot_data['pointNumber'].isnull().any() and shot_data['pointNumber'].is_monotonic_increasing:
    print("Check Passed: Point Numbers already exist")


# Check if pointNumber exists for every row and is not empty
if 'pointNumber' not in shot_data.columns or shot_data['pointNumber'].isnull().any():
    # Execute the script to calculate pointNumber
    point_starts = (shot_data['isPointStart'] == 1)
    shot_data['pointNumber'] = point_starts.cumsum()
    
    print("Data had missing point numbers. They were filled in automatically")
    
# Check if pointNumber is in increasing order
if not shot_data['pointNumber'].is_monotonic_increasing:
    # Print the condition failure if the 'pointNumber' column is not in increasing order
    print("Data had point numbers in the wrong order. They were filled in automatically")
    point_starts = (shot_data['isPointStart'] == 1)
    shot_data['pointNumber'] = point_starts.cumsum()
   

Data had missing point numbers. They were filled in automatically


In [28]:
missing_pointNumber_rows = shot_data[shot_data['pointNumber'].isnull()]

# Check if there are missing rows
if len(missing_pointNumber_rows) == 0:
    print("Check Passed: All rows have pointNumber")
else:
    print(f"Count of rows missing 'pointNumber': {len(missing_pointNumber_rows)}")

Check Passed: All rows have pointNumber


## Add Column:
#### player1Hand, player2Hand

In [30]:
# fillInplayer1Hand = "Right"
# fillInplayer2Hand = "Right"

# Check if player1Hand key exists in shot_data, if not, create it
if 'player1Hand' not in shot_data:
    shot_data['player1Hand'] = None

# Check if player2Hand key exists in shot_data, if not, create it
if 'player2Hand' not in shot_data:
    shot_data['player2Hand'] = None

if shot_data.at[0, 'player1Hand'] == None:
    print("player1Hand was blank. Filling with 'fillInplayer1Hand'")
    shot_data.at[0, 'player1Hand'] = fillInplayer1Hand
else:
    print("Check Passed: player1Hand is not blank. Current value:", shot_data.at[0, 'player1Hand'])

# Check if player2Hand is blank, if so, fill it with 'fillInplayer2Hand'
if shot_data.at[0, 'player2Hand'] == None:
    print("player2Hand was blank. Filling with 'fillInplayer2Hand'")
    shot_data.at[0, 'player2Hand'] = fillInplayer2Hand
else:
    print("Check Passed: player2Hand is not blank. Current value:", shot_data.at[0, 'player2Hand'])


print("Player1Hand Current value:", shot_data.at[0, 'player1Hand'])
print("Player2Hand Current value:", shot_data.at[0, 'player2Hand'])

Check Passed: player1Hand is not blank. Current value: right
Check Passed: player2Hand is not blank. Current value: right
Player1Hand Current value: right
Player2Hand Current value: right


### Filling Columns: Forward Fill
- 'gameScore'
- 'setScore'
- 'serverName'
- 'player1Name',
- 'player2Name'
- 'player2Hand'

# Forward Fill GameScore, SetScore, tiebreakScore, serverName, player1Name, player2Name

In [32]:
columns_to_fill = ['pointScore','gameScore','setScore', 'serverName', 'player1Name', 'player2Name', 'player1Hand', 'player2Hand'] 
# columns_to_fill = ['gameScore','setScore', 'tiebreakScore', 'serverName', 'player1Name', 'player2Name'] 
# Add player1Hand, player2Hand when it is in tagger

for column in columns_to_fill:
    shot_data[column].replace(['', 'na'], pd.NaT, inplace=True)
    shot_data[column] = shot_data[column].ffill()

## Error Check 7: 
#### Check for NA's

- all 0, except missing pointScore should match amount of tiebreak shots
- side can have missing, for old tagger and not tagging side of each shot

### Fixing:
- Open CSV in Google Sheets (Excel will change to date format), output game, set, point Score of missing values. Then find in google sheets, adjust, redownload, and re-upload into notebook

In [34]:
# Count empty strings in each column
empty_string_counts = (shot_data == "").sum()

# Filter out columns with zero empty strings
non_zero_counts = empty_string_counts[empty_string_counts > 0]

# Count NaN values in 'pointScore' column when 'gameScore' is '6-6'
na_tiebreak_count = shot_data.loc[shot_data['gameScore'] == '6-6', 'pointScore'].isna().sum()

side_na_count = shot_data.loc[shot_data['isPointStart'] == 1, 'side'].isna().sum()
print("\nCount of NaN values for 'side' on Point Start:", side_na_count)

# Display the count
print("\nCount of Nan when gameScore is '6-6' (# tiebreak shots):", na_tiebreak_count)


# Count NaN or empty values in specified columns
na_counts = shot_data[[ 'pointScore', 'shotInRally', 'gameScore', 'setScore', 'side', 'serverName']].isna().sum()

# Display the counts
print(f"\nCount of NA's in these columns\n{na_counts}")

# Display the counts
print("Count of empty strings in each column that includes at least one:")
print(non_zero_counts)




Count of NaN values for 'side' on Point Start: 0

Count of Nan when gameScore is '6-6' (# tiebreak shots): 0

Count of NA's in these columns
pointScore     0
shotInRally    0
gameScore      0
setScore       0
side           0
serverName     0
dtype: int64
Count of empty strings in each column that includes at least one:
Series([], dtype: int64)


## Error Check 8: 
####  rows where side is empty
- If side missing, focus on rows where (shotInRally == 1) and returns (shotInRally == 2 ), anything else can leave NA

In [36]:
print(shot_data[shot_data["side"].isnull()][['pointScore', 'gameScore', 'setScore','side', 'shotInRally']])

Empty DataFrame
Columns: [pointScore, gameScore, setScore, side, shotInRally]
Index: []


## Error Check 9: 
####  Outputs Missing shotInRally Rows

In [38]:
empty_shot_rows = shot_data[shot_data['shotInRally'].isnull()]

if not empty_shot_rows.empty:
    # Iterate over the index of empty_shot_rows
    for index in empty_shot_rows.index:
        # Get the row with empty shotInRally
        empty_row = shot_data.loc[index]
        
        # Get the row above it
        if index - 1 >= 0:
            above_row = shot_data.loc[index - 1]
        else:
            above_row = None
        
        # Get the row below it
        if index + 1 < len(shot_data):
            below_row = shot_data.loc[index + 1]
        else:
            below_row = None
        
        # Print the rows
        print("Empty row:")
        print(empty_row)
        print("Row above:")
        print(above_row)
        print("Row below:")
        print(below_row)
        print("====================")
else:
    print("Check Passed: No Missing shotInRally")


Check Passed: No Missing shotInRally


## Error Check 10: 
####  Check that pointNumber is increasing consecutively

- if video is bad and points are cut out, this can fail and that's ok.

In [40]:
point_numbers = shot_data['pointNumber'].unique()

# Check if the point numbers are consecutive
if list(point_numbers) == list(range(1, len(point_numbers) + 1)):
    print("Check Passed")
else:
    raise ValueError("Error: The 'pointNumber' column does not contain consecutive numbers starting from 1.")


Check Passed


## Error Check 11: 
#### Check same amount of start and end points

In [42]:
# Count of Point Start and Point End
num_point_start = shot_data['isPointStart'].sum()

# Count the number of rows where isPointEnd is equal to 1
num_point_end = shot_data['isPointEnd'].sum()

print("Number of rows with isPointStart = 1:", num_point_start)
print("Number of rows with isPointEnd = 1:", num_point_end)

Number of rows with isPointStart = 1: 130.0
Number of rows with isPointEnd = 1: 130.0


## Error Check 12: 
#### Output consecutive isPointStart with no isPointEnd, and vice versa for isPointEnd


In [44]:
# Shubhan

# Output total start, total end
# How many Points aren't enclosing between point start and point end 

# Count where isPointStart = 1
count_isPointStart = (shot_data['isPointStart'] == 1).sum()

# Count where isPointEnd = 1
count_isPointEnd = (shot_data['isPointEnd'] == 1).sum()

print("Total count where isPointStart = 1:", count_isPointStart)
print("Total count where isPointEnd = 1:", count_isPointEnd)

# Output Point/Game Scores where there is consecutive isPointStart with no isPointEnd
def check_isPointStart_in_between(shot_data):
    start_indices = shot_data.index[shot_data['isPointStart'] == 1]  # Get indices where isPointStart is True
    end_indices = shot_data.index[shot_data['isPointEnd'] == 1]  # Get indices where isPointEnd is True
    start_without_end = []

    for i in range(len(start_indices) - 1):
        current_start_index = start_indices[i]
        next_start_index = start_indices[i + 1]

        # Get all isPointEnd indices between current_start_index and next_start_index
        end_between_starts = end_indices[(end_indices >= current_start_index) & (end_indices < next_start_index)]

        # Check if there is at least one isPointEnd between current_start_index and next_start_index
        if len(end_between_starts) == 0:
            start_without_end.append(current_start_index)

    # Check the last isPointStart
    last_start_index = start_indices[-1]
    end_after_last_start = end_indices[end_indices >= last_start_index]

    if len(end_after_last_start) == 0:
        start_without_end.append(last_start_index)

    if start_without_end:
        print(" ")
        print("isPointStart without corresponding isPointEnd:")
        for index in start_without_end:
#             print("Index:", index)
#             print("pointScore:", shot_data.at[index, 'pointScore'])
#             print("gameScore:", shot_data.at[index, 'gameScore'])
#             print("setScore:", shot_data.at[index, 'setScore'])
            print(" ")
            print(shot_data.loc[start_without_end][['pointScore', 'gameScore', 'setScore', 'isPointStart']])
            print(" ")

        
        return False
    else:
        print("All isPointStart have corresponding isPointEnd.")
        return True

# Check if the condition is met
check_isPointStart_in_between(shot_data)

# Output Point/Game Scores where there is consecutive isPointEnd with no isPointStart
def check_isPointEnd_in_between(shot_data):
    start_indices = shot_data.index[shot_data['isPointStart'] == 1]  # Get indices where isPointStart is True
    end_indices = shot_data.index[shot_data['isPointEnd'] == 1]  # Get indices where isPointEnd is True
    end_without_start = []

    for i in range(len(end_indices) - 1):
        current_end_index = end_indices[i]
        next_end_index = end_indices[i + 1]

        # Get all isPointStart indices b/w current_end_index and next_end_index
        start_between_ends = start_indices[(start_indices > current_end_index) & (start_indices <= next_end_index)]

        # Check if there is at least one isPointStart between current_end_index and next_end_index
        if len(start_between_ends) == 0:
            end_without_start.append(current_end_index)

    # NOTE: This will always append the last index of the sheet because the last end will never have a corresponding start so I'm not sure if its necessary to add it to the end_without_start array
    # Check the last isPointStart
    """
    last_end_index = end_indices[-1]
    start_after_last_end = start_indices[start_indices >= last_end_index]

    if len(start_after_last_end) == 0:
        end_without_start.append(last_end_index)
    """

    if end_without_start:
        print("isPointEnd without corresponding isPointStart:")
#         for index in end_without_start:
#             print("Index:", index)
#             print("pointScore:", shot_data.at[index, 'pointScore'])
#             print("gameScore:", shot_data.at[index, 'gameScore'])
#             print("setScore:", shot_data.at[index, 'setScore'])
#             print("setScore:", shot_data.at[index, 'tiebreakScore'])

        # Output the rows of shot_data that correspond to the indices in end_without_start
        print(" ")
        print(shot_data.loc[end_without_start][['pointScore', 'gameScore', 'setScore', 'isPointEnd']])

        return False
    else:
        print("All isPointEnd have corresponding isPointStart.")
        return True

# Check if the condition is met
check_isPointEnd_in_between(shot_data)

Total count where isPointStart = 1: 130
Total count where isPointEnd = 1: 130
All isPointStart have corresponding isPointEnd.
All isPointEnd have corresponding isPointStart.


True

## Error Check 14:
#### Change the score from incorrect date format
- accounts for all variations of dates eg. (0-00-0000, 0/0/0000)

In [46]:
## Error Check 13:
#### Make Jan-00 back into 1-0 for Game/Set Score

# Make Scores Strings not Date Time
columns_to_convert = ['gameScore', 'setScore'] #if no tiebreakScore
# columns_to_convert = ['gameScore', 'setScore', 'tiebreakScore']
shot_data[columns_to_convert] = shot_data[columns_to_convert].astype(object)

In [47]:
import re

# Define a mapping for month abbreviations
month_mapping = {'Jan': '1', 'Feb': '2', 'Mar': '3', 'Apr': '4', 'May': '5', 'Jun': '6',
                 'Jul': '7', 'Aug': '8', 'Sep': '9', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

# Function to convert string like 'Jan-00' to '1-0'
def convert_score_string(score_str):
    # Check if the string has a month abbreviation and a year ending with '00'
    if re.match(r'^\d{1,2}-[A-Za-z]{3}$', score_str):
        # Extract year and month abbreviation
        year, month = score_str.split('-')

        # Remove leading zeros from the year
        year = str(int(year))

        # Replace month abbreviation with corresponding number
        month_number = month_mapping.get(month, month)

        # Concatenate the parts to form the transformed string
        transformed_str = f'{year}-{month_number}'
        return transformed_str

    # Check if the string has a month abbreviation and a year with leading '0's
    elif re.match(r'^[A-Za-z]{3}-\d{1,2}$', score_str):
        # Extract month abbreviation and year
        month, year = score_str.split('-')

        # Replace month abbreviation with corresponding number
        month_number = month_mapping.get(month, month)

        # Remove leading zeros from the year
        year = str(int(year))

        # Concatenate the parts to form the transformed string
        transformed_str = f'{month_number}-{year}'
        return transformed_str

    # Check if the string has a date in the format 'month/day/year'
    elif re.match(r'^\d{1,2}/\d{1,2}/\d{4}$', score_str):
        # Extract month, day, and year
        month, day, year = score_str.split('/')

        # Remove leading zeros from month and day
        month = str(int(month))
        day = str(int(day))

        # Concatenate the parts to form the transformed string
        transformed_str = f'{month}-{day}'
        return transformed_str

    # Check if the string has a date in the format 'month-day-year'
    elif re.match(r'^\d{1,2}-\d{1,2}-\d{4}$', score_str):
        # Extract month, day, and year
        month, day, year = score_str.split('-')

        # Remove leading zeros from month and day
        month = str(int(month))
        day = str(int(day))

        # Concatenate the parts to form the transformed string
        transformed_str = f'{month}-{day}'
        return transformed_str

    return score_str

# Apply the conversion function to the relevant columns in shot_data
shot_data['gameScore'] = shot_data['gameScore'].apply(convert_score_string)
shot_data['setScore'] = shot_data['setScore'].apply(convert_score_string)
# shot_data['tiebreakerScore'] = shot_data['tiebreakerScore'].apply(convert_score_string)


## Error Check 15: 
#### Check that there is no strings in set, game scores. 

In [49]:
# Assuming shot_data is your DataFrame
unique_set_scores = shot_data['setScore'].unique()
unique_game_scores = shot_data['gameScore'].unique()

# Print unique values
print("Unique Set Scores:", unique_set_scores)
print("Unique Game Scores:", unique_game_scores)


Unique Set Scores: ['0-0' '1-0']
Unique Game Scores: ['0-0' '1-0' '2-0' '2-1' '3-1' '4-1' '4-2' '4-3' '5-3' '5-4' '5-5' '5-6'
 '6-6' '0-1' '3-0' '5-2']


## Error Check 16 : 
### Print out all the rows where isPointStart time doesn't start in the beginning of a rally

In [51]:
# Filter the DataFrame
filtered_rows = shot_data[(shot_data['isPointStart'] == 1) & (shot_data['shotInRally'] != 1)]

# Print the filtered rows
if filtered_rows.empty:
    print ("Check passed.")
else:
    print("Rows where isPointStart = 1 and shotInRally != 1:")
    print(filtered_rows)


Check passed.


## Error Check 17: 
### Find the rows where isPointEnd = 1 and shotInRally = 1 but is neither an ace or a double fault

In [53]:
# Filter the DataFrame
filtered_rows = shot_data[
    (shot_data['isPointEnd'] == 1) & 
    (shot_data['shotInRally'] == 1) & 
    (shot_data['firstServeIn'] != 0) &
    (shot_data['secondServeIn'] != 0) &
    (shot_data['isAce'] != 1)
]

# Print the filtered rows
if filtered_rows.empty:
    print ("Check passed.")
else:
    print("Rows where isPointEnd = 1, shotInRally = 1, firstServeIn and secondServeIn are both not 0, and isAce != 1:")
    print(filtered_rows)

Check passed.


## Error Check 18: 
### Find all the rows where there are duplicate isPointStarts == 1 points
- Finds all the points where the there are the same score and isPointStart == 1

In [55]:
# Filter the DataFrame
filtered_rows = shot_data[shot_data['isPointStart'] == 1]

# Output rows where isPointStart is 1 and pointScore, gameScore, and setScore have the same value
output_rows = filtered_rows[filtered_rows.duplicated(subset=['pointScore', 'gameScore', 'setScore'], keep=False)]

# Print the output rows
if output_rows.empty:
    print ("Check passed.")
else:
    print("Rows where isPointStart is 1 and pointScore, gameScore, and setScore have the same value:")
    print(output_rows[['pointScore', 'gameScore', 'setScore', 'tiebreakScore', 'isPointStart']])



Check passed.


## Error Check 19: 
### Find all the rows where there are duplicate isPointEnd == 1 points
- Finds all the points where the there are the same score and isPointEnd == 1

In [57]:
# Filter the DataFrame
filtered_rows = shot_data[shot_data['isPointEnd'] == 1]

# Output rows where isPointStart is 1 and pointScore, gameScore, and setScore have the same value
output_rows = filtered_rows[filtered_rows.duplicated(subset=['pointScore', 'gameScore', 'setScore'], keep=False)]

# Print the output rows
if output_rows.empty:
    print ("Check passed.")
else:
    print("Rows where isPointEnd is 1 and pointScore, gameScore, and setScore have the same value:")
    print(output_rows[['pointScore', 'gameScore', 'setScore', 'isPointStart']])


Check passed.


## Error Check 20:
### Find all the rows where there is isPointEnd == 1 but there is no isWinner, isErrorWideL, isErrorWideR, isErrorNet, isErrorLong


In [59]:
filtered_rows = shot_data[(shot_data['isPointEnd'] == 1) &
                          (shot_data['isWinner'] != 1) &
                          (shot_data['isErrorWideL'] != 1) &
                          (shot_data['isErrorWideR'] != 1) &
                          (shot_data['isErrorNet'] != 1) & 
                          (shot_data['isErrorLong'] != 1) &
                          (shot_data['firstServeIn'] != 0) & 
                          (shot_data['secondServeIn'] != 0)]

if filtered_rows.empty:
    print ("Check passed.")
else:
    print("Error: isPointEnd == 1 but there is no point end shot")
    print(filtered_rows[['pointScore', 'gameScore', 'setScore','isPointStart', 'isPointEnd', 'isWinner', 'isErrorWideL', 'isErrorWideR', 'isErrorNet', 'isErrorLong']])


Check passed.


# Add More Columns

### Add Column: tiebreakScore
#### WARNING: ONLY RUN THIS CELL ONCE!!!

In [62]:
# shot_data['tiebreakScore'] = np.nan

# # Function to reverse pointScore
# def reverse_point_score(score):
#     if '-' in score:
#         parts = score.split('-')
#         return '-'.join(parts[::-1])
#     return score

# # Apply reverse_point_score function where serverName is 'Player2'
# shot_data.loc[shot_data['serverName'] == 'Player2', 'pointScore'] = \
#     shot_data.loc[shot_data['serverName'] == 'Player2', 'pointScore'].apply(reverse_point_score)

# # Transfer values from pointScore to tiebreakScore where gameScore is '6-6'
# shot_data.loc[shot_data['gameScore'] == '6-6', 'tiebreakScore'] = shot_data['pointScore']

In [63]:
shot_data['tiebreakScore'] = np.nan


player2_new_name = shot_data['player2Name'].iloc[0]

# Function to reverse pointScore
def reverse_point_score(score):
    if '-' in score:
        parts = score.split('-')
        return '-'.join(parts[::-1])
    return score

# Apply reverse_point_score function where serverName is 'Player2'
shot_data.loc[shot_data['serverName'] == player2_new_name, 'pointScore'] = \
    shot_data.loc[shot_data['serverName'] == player2_new_name, 'pointScore'].apply(reverse_point_score)

# Transfer values from pointScore to tiebreakScore where gameScore is '6-6'
shot_data.loc[shot_data['gameScore'] == '6-6', 'tiebreakScore'] = shot_data['pointScore']

# Set the pointScore to NaN where tiebreakScore is not NaN
shot_data.loc[pd.notna(shot_data['tiebreakScore']), 'pointScore'] = np.nan


/var/folders/s6/qrny1b0d5p14mk156f_g7lhm0000gn/T/ipykernel_11365/3857377149.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0-0' '0-0' '0-0' '0-0' '0-0' '1-0' '1-0' '2-0' '2-0' '3-0' '3-0' '3-0'
 '3-0' '3-0' '3-0' '3-0' '3-0' '3-1' '3-1' '3-1' '4-1' '4-1' '4-1' '4-1'
 '4-1' '5-1' '5-1' '5-1' '5-1' '6-1' '6-1' '6-1' '6-1' '6-1']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  shot_data.loc[shot_data['gameScore'] == '6-6', 'tiebreakScore'] = shot_data['pointScore']


In [64]:
shot_data.loc[shot_data['gameScore'] == '6-6', 'tiebreakScore']

300    0-0
301    0-0
302    0-0
303    0-0
304    0-0
305    1-0
306    1-0
307    2-0
308    2-0
309    3-0
310    3-0
311    3-0
312    3-0
313    3-0
314    3-0
315    3-0
316    3-0
317    3-1
318    3-1
319    3-1
320    4-1
321    4-1
322    4-1
323    4-1
324    4-1
325    5-1
326    5-1
327    5-1
328    5-1
329    6-1
330    6-1
331    6-1
332    6-1
333    6-1
Name: tiebreakScore, dtype: object

In [65]:
shot_data.loc[shot_data['gameScore'] == '6-6', 'pointScore'].head()

300    NaN
301    NaN
302    NaN
303    NaN
304    NaN
Name: pointScore, dtype: object

### Add Column: returnerName

In [67]:
# Get Returner Name for Shot DF

# Extract player1Name and player2Name from the first row of shot_data
first_row_shot_df = shot_data.iloc[0]
player1_name = first_row_shot_df['player1Name']
player2_name = first_row_shot_df['player2Name']

# # Can Also Manually inpute Player 1 and 2 name
# player1_name = "Kimmi Hance"
# player2_name = "Malaika Rapolu"

def get_returner_name(server_name):
    return player2_name if server_name == player1_name else player1_name

# Add 'Returner Name' column to point_df using the function
shot_data['returnerName'] = shot_data['serverName'].apply(get_returner_name)

print(f"Player 1 = {player1_name}, Player 2 = {player2_name}")
# print(shot_data[['serverName','returnerName']])

Player 1 = Elise Wagle, Player 2 = Motka Komac


### Add Column: shotHitBy

In [69]:
# Add Shot Hit By for each Shot

shot_data['shotHitBy'] = shot_data.apply(lambda row: row['serverName'] if row['shotInRally'] % 2 == 1 else row['returnerName'], axis=1)
# print(shot_data[['serverName', 'shotHitBy','pointNumber', 'shotInRally']].head(20))

### Add Column: isInsideOut, isInsideIn

In [72]:
# Add columns for isInsideOut and isInsideIn, initially set to 0
shot_data['isInsideOut'] = None
shot_data['isInsideIn'] = None

# Iterate through rows
for index, row in shot_data.iterrows():
    shotHitBy = row['shotHitBy']
    player_hand = row['player1Hand'] if shotHitBy == row['player1Name'] else row['player2Hand']
    

    
    if player_hand == "Right":
        if row['side'] == "Deuce" and row['shotFhBh'] == "Backhand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Ad" and row['shotFhBh'] == "Forehand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Deuce" and row['shotFhBh'] == "Backhand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
        elif row['side'] == "Ad" and row['shotFhBh'] == "Forehand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
    elif player_hand == "Left":
        if row['side'] == "Ad" and row['shotFhBh'] == "Backhand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Deuce" and row['shotFhBh'] == "Forehand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Ad" and row['shotFhBh'] == "Backhand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
        elif row['side'] == "Deuce" and row['shotFhBh'] == "Forehand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1

### Add Column: isAce

In [74]:
# Add the Ace column
shot_data['isAce'] = None

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if (row['firstServeIn'] == 1 or row['secondServeIn'] == 1): # either first or second serve went in
                shot_data.at[index, 'isAce'] = 1

### Add Column: isDoubleFault

In [76]:
# Add the DoubleFault column
shot_data['isDoubleFault'] = None

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if (row['firstServeIn'] != 1 and row['secondServeIn'] != 1): # either first or second serve went in
                shot_data.at[index, 'isDoubleFault'] = 1

### Add Column: pointWonBy, lastShotError

In [78]:
# Add the 'pointWonBy' column
shot_data['pointWonBy'] = None

# Add the 'lastShotError' column
shot_data['lastShotError'] = 0

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if row['isAce'] == 1: 
                shot_data.at[index, 'pointWonBy'] = row['serverName']
            elif row['isDoubleFault'] == 1: 
                shot_data.at[index, 'pointWonBy'] = row['returnerName']

                
        elif row['shotInRally'] != 1:
            if row['isErrorWideR'] == 1 or row['isErrorWideL'] == 1 or row['isErrorNet'] == 1 or row['isErrorLong'] == 1: # if error
                shot_data.at[index, 'lastShotError'] = 1
                
                if row['shotInRally'] % 2 == 0:
                    shot_data.at[index, 'pointWonBy'] = row['serverName']
                else:
                    shot_data.at[index, 'pointWonBy'] = row['returnerName']
        
            elif row['isWinner'] == 1:
                if row['shotInRally'] % 2 == 0:
                    shot_data.at[index, 'pointWonBy'] = row['returnerName']
                else:
                    shot_data.at[index, 'pointWonBy'] = row['serverName']


## Error Check 21: 
#### Output Point End with no pointWonBy


In [80]:
# Missing Point End
print(shot_data[(shot_data['isPointEnd'] == 1) & (shot_data['pointWonBy'].isnull())][['pointScore', 'gameScore','setScore', 'lastShotError', 'isWinner', 'isErrorWideR', 'isErrorWideL',
       'isErrorNet', 'isErrorLong', 'pointWonBy', 'serverName', 'shotInRally']])


Empty DataFrame
Columns: [pointScore, gameScore, setScore, lastShotError, isWinner, isErrorWideR, isErrorWideL, isErrorNet, isErrorLong, pointWonBy, serverName, shotInRally]
Index: []


### Backward Fill: PointWonBy

In [82]:
shot_data['pointWonBy'].bfill()

0      Motka Komac
1      Motka Komac
2      Motka Komac
3      Motka Komac
4      Motka Komac
          ...     
680    Elise Wagle
681    Elise Wagle
682    Elise Wagle
683    Elise Wagle
684    Elise Wagle
Name: pointWonBy, Length: 685, dtype: object

## (CAITLIN) Error Check 22: 
#### Find all rows with mismatched serve in/serve zone


In [84]:
# Find indices of all rows with firstServeIn data
first_serve_row = shot_data.index[shot_data['firstServeIn'].notnull()]
# Find indices of all rows with firstServeZone data
first_serve_placement_row = shot_data.index[shot_data['firstServeZone'].notnull()]
# Check which indices are not found in one column but are found in the other
mismatched_first_serve_rows = list(set(first_serve_row).difference(first_serve_placement_row))
# Add 2 to the indices to match numbering of Google Sheets
mismatched_first_serve_rows = [x+2 for x in mismatched_first_serve_rows]
if not mismatched_first_serve_rows:
    print("Check passed for first serves.")
else:
    print("Rows where first serve in and first serve zone are not found together: " + str(mismatched_first_serve_rows))

second_serve_row = shot_data.index[shot_data['secondServeIn'].notnull()]
second_serve_placement_row = shot_data.index[shot_data['secondServeZone'].notnull()]
mismatched_second_serve_rows = list(set(second_serve_row).difference(first_serve_placement_row))
mismatched_second_serve_rows = [x+2 for x in mismatched_second_serve_rows]
if not mismatched_second_serve_rows:
    print("Check passed for second serves.")
else:
    print("Rows where second serve in and second serve zone are not found together: " + str(mismatched_second_serve_rows))

Check passed for first serves.
Check passed for second serves.


### Add Column: serveResult, serveInPlacement

In [86]:
conditions = [
    (shot_data['isPointStart'] == 1) & (shot_data['firstServeIn'] == 1),
    (shot_data['isPointStart'] == 1) & (shot_data['firstServeIn'] != 1) & (shot_data['secondServeIn'] == 1),
    (shot_data['isPointStart'] == 1) & (shot_data['firstServeIn'] != 1) & (shot_data['secondServeIn'] != 1),]

# Define the values to be assigned for each condition
values_result = ['1st Serve In', '2nd Serve In', 'Double Fault']
values_placement = [shot_data['firstServeZone'], shot_data['secondServeZone'], shot_data['secondServeZone']]

# Use numpy.select to assign values based on conditions
shot_data['serveResult'] = np.select(conditions, values_result, default='')
shot_data['serveInPlacement'] = np.select(conditions, values_placement, default='')

# Now shot_data DataFrame is updated with serveResult and serveInPlacement values where isPointStart is 1
# print(shot_data[['serveResult', 'serveInPlacement','pointScore','side']].head(60))

In [87]:
shot_data.replace("", None, inplace=True)

In [88]:
shot_data.loc[shot_data['serveResult'] == 'Double Fault', 'serveInPlacement']


25     Body
109    Body
110    Body
111       T
123       T
129       T
166       T
167    Body
339    Body
352    Body
378    Wide
390       T
428    Body
Name: serveInPlacement, dtype: object

### Add Column: Depth

In [90]:
def depth_metric(shotInRally, x, y, side) :
    
    if (x >= -157.5) & (x <= 157.5):
    
        if side == 'Near':
            if shotInRally % 2 == 0:
                if -455 < y < -350: return 'Deep'
                if -350 < y < 0: return 'Short'
                if y < -455: return 'Long'

            elif shotInRally % 2 == 1:
                if 455 > y > 350: return 'Deep'
                if 0 < y < 350: return 'Short'
                if y > 455: return 'Long'

        elif side == 'Far':
            if shotInRally % 2 == 1:
                if -455 < y < -350: return 'Deep'
                if -350 < y < 0: return 'Short'
                if y < -455: return 'Long'

            elif shotInRally % 2 == 0:
                if 455 > y > 350: return 'Deep'
                if 0 < y < 350: return 'Short'
                if y > 455: return 'Long'
        
shot_data['depth'] = shot_data.apply(lambda x: depth_metric(x['shotInRally'], x['shotLocationX'], x['shotLocationY'], x['serverFarNear']), axis=1)

In [91]:
shot_data[['shotInRally','serverFarNear', 'shotLocationY', 'depth']]['depth'].value_counts()

depth
Short    307
Deep     151
Long      15
Name: count, dtype: int64

### Add Column: isAprroach

# Double Check BEFORE Outputting

In [94]:
print(shot_data.head(10))

  pointScore gameScore setScore  isPointStart  pointStartTime  isPointEnd  \
0        0-0       0-0      0-0           1.0            6749         NaN   
1        0-0       0-0      0-0           NaN           10039         NaN   
2        0-0       0-0      0-0           NaN           11629         NaN   
3        0-0       0-0      0-0           NaN           13157         NaN   
4        0-0       0-0      0-0           NaN           14863         NaN   
5        0-0       0-0      0-0           NaN           16233         NaN   
6        0-0       0-0      0-0           NaN           18230         NaN   
7        0-0       0-0      0-0           NaN           19945         NaN   
8        0-0       0-0      0-0           NaN           21123         NaN   
9        0-0       0-0      0-0           NaN           22522         NaN   

   pointEndTime  pointNumber  isBreakPoint  shotInRally  ... returnerName  \
0           NaN            1           NaN            1  ...  Elise Wagle  

In [95]:
print(shot_data.tail(10))

    pointScore gameScore setScore  isPointStart  pointStartTime  isPointEnd  \
675      30-15       5-3      1-0           NaN         6105993         NaN   
676      30-15       5-3      1-0           NaN         6107535         NaN   
677      30-15       5-3      1-0           NaN         6109787         NaN   
678      30-15       5-3      1-0           NaN         6111326         NaN   
679      30-15       5-3      1-0           NaN         6112340         1.0   
680      30-30       5-3      1-0           1.0         6113969         NaN   
681      30-30       5-3      1-0           NaN         6145675         1.0   
682      40-30       5-3      1-0           1.0         6148559         NaN   
683      40-30       5-3      1-0           NaN         6192885         NaN   
684      40-30       5-3      1-0           NaN         6194703         1.0   

     pointEndTime  pointNumber  isBreakPoint  shotInRally  ... returnerName  \
675           NaN          128           NaN       

# Output ShotCSV

In [97]:
# # # Ouput Improved Shot Csv HERE
# # Assuming point_df is your DataFrame and player1Name and player2Name are the names from the first row
player1NameNoSpace = shot_data.iloc[0]['player1Name'].replace(" ", "")
player2NameNoSpace = shot_data.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
shot_data.to_csv(f'Shot_Visuals_{player1NameNoSpace}_{player2NameNoSpace}.csv', index=False)

In [98]:
shot_data['pointNumber'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130])

# Below is for Point CSV

### Create Point DF

In [101]:
# Creating point_df (with only 1 row for each pointNumber)
point_df = shot_data.drop_duplicates(subset='pointNumber')[['pointNumber']]
# point_df.shape

### Add Column: player1Name, player2Name
### Set Variable: first_player1Name, first_player2Name

In [103]:
# Extract the first value of player1Name and player2Name from shot_data
first_player1Name = shot_data['player1Name'].iloc[0]
first_player2Name = shot_data['player2Name'].iloc[0]

# Fill in the first value into all rows of point_df['player1Name'] and point_df['player2Name']
point_df['player1Name'] = first_player1Name
point_df['player2Name'] = first_player2Name

### Add Column: Scores

In [105]:
point_df['pointScore'] = shot_data.groupby('pointNumber')['pointScore'].first().values
point_df['gameScore'] = shot_data.groupby('pointNumber')['gameScore'].first().values
point_df['setScore'] = shot_data.groupby('pointNumber')['setScore'].first().values
point_df['tiebreakScore'] = shot_data.groupby('pointNumber')['tiebreakScore'].first().values

### Add Column: Side

In [107]:
# Group shot_data by 'pointNumber' and get the first 'side' value for each group
side_values = shot_data.groupby('pointNumber')['side'].first().reset_index()

point_df['side'] = side_values['side'].values

### Add Column: serverName, returnerName

In [109]:
# Adds Server and Returner Names and pointScore

point_df['serverName'] = shot_data.groupby('pointNumber')['serverName'].first().values
point_df['returnerName'] = shot_data.groupby('pointNumber')['returnerName'].first().values


In [110]:
client_team_value = shot_data.loc[0, 'clientTeam']
opponent_team_value = shot_data.loc[0, 'opponentTeam']

# Adding 'clientTeam' and 'opponentTeam' columns to point_df
point_df['clientTeam'] = client_team_value
point_df['opponentTeam'] = opponent_team_value


## Warning: Will be Empty if Timestamp separate

In [112]:
# Add Start and End times per point

for index, row in shot_data.iterrows():
    point_number = row['pointNumber']
    
    if row['isPointStart'] == 1:
        point_df.loc[point_df['pointNumber'] == point_number, 'Position'] = row['pointStartTime']
    if row['isPointEnd'] == 1:
        point_df.loc[point_df['pointNumber'] == point_number, 'pointEndPosition'] = row['pointEndTime']

# Add Duration
point_df['Duration'] = point_df['pointEndPosition'] - point_df['Position']

In [113]:
len(point_df['Position'])

130

In [114]:
len(point_df['pointEndPosition'])
# point_df['Duration']

130

### Add Column: Rally Column
- rallyCount (x amount)
- rallyCountFreq (1-4,5-8,9-12,13+)

In [116]:
# Find the highest shotInRally for each pointNumber in shot_data
max_rally_per_point = shot_data.groupby('pointNumber')['shotInRally'].max().reset_index()
point_df['rallyCount'] = list(max_rally_per_point['shotInRally'])

# Add 'rallyCountFreq' column based on specified conditions
point_df['rallyCountFreq'] = point_df['rallyCount'].apply(lambda x: '1 - 4' if 1 <= x <= 4 else ('5 - 8' if 5 <= x <= 8 else ('9 - 12' if 9 <= x <= 12 else ('13 +' if x >= 13 else 'Error'))))


### Add Column: Serve 
- firstServeIn
- secondServeIn
- serveResult
- serveInPlacement

Part 2
- serveResult
- serveInPlacement
 
Part 3
- firstServeZone
- secondServeZone

In [118]:
# Add firstServeIn and secondServeIn

# Add firstServeIn and secondServeIn columns
point_df['firstServeIn'] = 0
point_df['secondServeIn'] = 0

for point_number in shot_data['pointNumber'].unique():
    # Check if firstServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['firstServeIn'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'firstServeIn'] = 1
    
    # Check if secondServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['secondServeIn'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'secondServeIn'] = 1

#part 2
start_points = shot_data[shot_data['isPointStart'] == 1]

# Set values in point_df to corresponding values from start_points
point_df['serveResult'] = start_points['serveResult'].values
point_df['serveInPlacement'] = start_points['serveInPlacement'].values

# part 3
serve_zones = shot_data.loc[shot_data['shotInRally'] == 1, ['pointNumber', 'firstServeZone', 'secondServeZone', 'firstServeIn', 'secondServeIn']].drop_duplicates()
point_df['firstServeZone'] = shot_data.groupby('pointNumber')['firstServeZone'].first().values
point_df['secondServeZone'] = shot_data.groupby('pointNumber')['secondServeZone'].first().values

### Add Column: Ace

In [120]:
point_df['isAce'] = ((point_df['rallyCount'] == 1) & ((point_df['serveResult'] != "Double Fault")))

# Display the resulting DataFrame
print(point_df[['pointNumber', 'rallyCount', 'isAce']].head(14))

    pointNumber  rallyCount  isAce
0             1          11  False
11            2           5  False
16            3           9  False
25            4           1  False
26            5           4  False
30            6           2  False
32            7           7  False
39            8           4  False
43            9           2  False
45           10           4  False
49           11           1   True
50           12           3  False
53           13           7  False
60           14           5  False


### Add Column: Server Coordinate Data

In [122]:
point_df['serverFarNear'] = shot_data.groupby('pointNumber')['serverFarNear'].first().values

# Add first serve X and Y column
point_df['firstServeXCoord'] = shot_data.groupby('pointNumber')['firstServeXCoord'].first().values
point_df['firstServeYCoord'] = shot_data.groupby('pointNumber')['firstServeYCoord'].first().values

# Add second serve X and Y column
point_df['secondServeXCoord'] = shot_data.groupby('pointNumber')['secondServeXCoord'].first().values
point_df['secondServeYCoord'] = shot_data.groupby('pointNumber')['secondServeYCoord'].first().values


point_df[['serverFarNear', 'firstServeXCoord', 'firstServeYCoord', 'secondServeXCoord', 'secondServeYCoord']]

,serverFarNear,firstServeXCoord,firstServeYCoord,secondServeXCoord,secondServeYCoord
0,Near,-72.0,181.0,NaN,NaN
11,Near,91.0,235.0,NaN,NaN
16,Near,-56.0,2.0,-117.0,235.0
25,Near,0.0,283.0,88.0,0.0
26,Near,-48.0,235.0,NaN,NaN
...,...,...,...,...,...
667,Far,-99.0,-103.0,NaN,NaN
669,Far,80.0,-161.0,NaN,NaN
672,Far,103.0,2.0,65.0,200.0
680,Far,48.0,-233.0,NaN,NaN


### Add Column: Return

In [124]:
# print(shot_data[shot_data['shotInRally'] == 2][['shotDirection','shotFhBh','pointScore','gameScore','setScore']])

In [125]:
# Set the initial values of 'returnDirection' and 'returnHand' columns to None
point_df['returnDirection'] = None
point_df['returnFhBh'] = None

# Iterate through pointNumber in shot_data
for point_number in shot_data['pointNumber'].unique():
    # Check if shotInRally == 2 exists for the given pointNumber
    if 2 in shot_data.loc[shot_data['pointNumber'] == point_number, 'shotInRally'].values:
        # Get the information from the corresponding row
        row_with_return_info = shot_data[(shot_data['pointNumber'] == point_number) & (shot_data['shotInRally'] == 2)].iloc[0]

        # Assign values to 'returnDirection' and 'returnHand' columns
        point_df.loc[point_df['pointNumber'] == point_number, 'returnDirection'] = row_with_return_info['shotDirection']
        point_df.loc[point_df['pointNumber'] == point_number, 'returnFhBh'] = row_with_return_info['shotFhBh']

# Display the modified DataFrame
print(point_df.head(10))

    pointNumber  player1Name  player2Name pointScore gameScore setScore  \
0             1  Elise Wagle  Motka Komac        0-0       0-0      0-0   
11            2  Elise Wagle  Motka Komac       0-15       0-0      0-0   
16            3  Elise Wagle  Motka Komac      15-15       0-0      0-0   
25            4  Elise Wagle  Motka Komac      15-30       0-0      0-0   
26            5  Elise Wagle  Motka Komac      30-30       0-0      0-0   
30            6  Elise Wagle  Motka Komac      40-30       0-0      0-0   
32            7  Elise Wagle  Motka Komac      40-40       0-0      0-0   
39            8  Elise Wagle  Motka Komac        0-0       1-0      0-0   
43            9  Elise Wagle  Motka Komac       15-0       1-0      0-0   
45           10  Elise Wagle  Motka Komac       30-0       1-0      0-0   

   tiebreakScore   side   serverName returnerName  ... firstServeZone  \
0           None  Deuce  Motka Komac  Elise Wagle  ...           Body   
11          None     Ad  Mot

### Add Column: Error Column
- errorType

In [127]:
# Create an empty DataFrame to store the results
error_results = pd.DataFrame(columns=['errorType', 'pointNumber'])

# Iterate through entire shot_data
for index, row in shot_data.iterrows():
    pointNumber = row['pointNumber']
    point_error_value = None
    
    if row['isErrorWideR'] == 1:
        point_error_value = 'Wide Right'
    elif row['isErrorWideL'] == 1:
        point_error_value = 'Wide Left'
    elif 'isErrorNet' in row and row['isErrorNet'] == 1:
        point_error_value = 'Net'
    elif row['isErrorLong'] == 1:
        point_error_value = 'Long'
    

    # If an error is found, append the result to the error_results DataFrame
    if point_error_value is not None:
        error_results = pd.concat([error_results, pd.DataFrame({'pointNumber': [pointNumber], 'errorType': [point_error_value]})], ignore_index=True)


# Drop duplicates based on 'pointNumber'
error_results = error_results.drop_duplicates(subset=['pointNumber'])

In [128]:
# Create a dictionary mapping 'pointNumber' to 'errorType' in error_results
error_type_mapping = dict(zip(error_results['pointNumber'], error_results['errorType']))

# Create 'errorType' column in point_df based on the mapping
point_df['errorType'] = point_df['pointNumber'].map(error_type_mapping)

point_df = point_df.replace({np.nan: None})

### Add Column: Error (Returns)

In [130]:
def get_return_error(row):
    if row['rallyCount'] == 2:
        return row['errorType']
    else:
        return None

point_df.loc[point_df['pointNumber'] == point_number, 'serveInPlacement'] = shot_data['secondServeZone']  

# Apply the functions to create the new columns
point_df['returnError'] = point_df.apply(get_return_error, axis=1)


### Add Column: Last Shot

In [132]:
point_df['lastShotDirection'] = None
point_df['lastShotFhBh'] = None
point_df['lastShotHitBy'] = None  
point_df['lastShotResult'] = None  

# Iterate through unique pointNumbers in shot_data
for point_number in shot_data['pointNumber'].unique():
    # Check if isPointEnd == 1 exists for the given pointNumber
    if 1 in shot_data.loc[shot_data['pointNumber'] == point_number, 'isPointEnd'].values:
        # Get the information from the corresponding row
        row_with_lastshot_info = shot_data[(shot_data['pointNumber'] == point_number) & (shot_data['isPointEnd'] == 1)].iloc[0]

        # Assign values to 'lastShotDirection' and 'lastShotFhBh' columns
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotDirection'] = row_with_lastshot_info['shotDirection']
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotFhBh'] = row_with_lastshot_info['shotFhBh']
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotHitBy'] = row_with_lastshot_info['shotHitBy']
        
        # Determine lastShotResult based on conditions
        if row_with_lastshot_info['isWinner'] == 1 and not row_with_lastshot_info['isAce']:
            point_df.loc[point_df['pointNumber'] == point_number, 'lastShotResult'] = "Winner"
        elif row_with_lastshot_info['lastShotError'] == 1:
            point_df.loc[point_df['pointNumber'] == point_number, 'lastShotResult'] = "Error"

In [133]:
print(point_df['lastShotResult'].unique())

['Winner' 'Error' None]


### Add Column: pointWonBy

In [135]:
# Initialize variables to keep track of the state
prev_point_number = None
point_won_by_list = []

# Iterate through the DataFrame
for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        # Check if pointNumber is different and consecutively increasing
        if prev_point_number is None or row['pointNumber'] == prev_point_number + 1:
            # Append pointWonBy to the list
            point_won_by_list.append(row['pointWonBy'])
            prev_point_number = row['pointNumber']
        else:
            raise ValueError("Error: Point numbers are not different or consecutively increasing.")
            break

# Add point_won_by_list as a new column to point_df
point_df['pointWonBy'] = point_won_by_list

### Add Column: isExcitingPoint

In [137]:
point_df['isExcitingPoint'] = shot_data.groupby('pointNumber')['isExcitingPoint'].count()

## Error Check 22: 
#### player1, player2, serverName, returnerName do not have misspellings

In [139]:
print(point_df['player1Name'].unique())
print(point_df['player2Name'].unique())
print(point_df['serverName'].unique())
print(point_df['returnerName'].unique())

if (len(point_df['serverName'].unique()) != 2):
    raise ValueError('Error: Mispelled Names')
    
if (len(point_df['returnerName'].unique()) != 2):
    raise ValueError('Error: Mispelled Names')

['Elise Wagle']
['Motka Komac']
['Motka Komac' 'Elise Wagle']
['Elise Wagle' 'Motka Komac']


## Error Check 23: 
#### pointWonBy has value that is not one of the two player names
- References: first_player1Name, first_player2Name from above server info in point_df

In [141]:
print(point_df['pointWonBy'].unique())
none_pointWonBy_df = point_df[~point_df['pointWonBy'].isin([first_player1Name, first_player2Name])]

if none_pointWonBy_df.empty:
    print("Check passed.")
else:
    print("Error: Rows where pointWonBy is not by either player!")
    print(none_pointWonBy_df)

['Motka Komac' 'Elise Wagle']
Check passed.


### Add Column: Break Point

In [143]:
break_point_values = ['0-40', '15-40', '30-40', '40-40']

# Create 'isBreakPoint' column in point_df
point_df['isBreakPoint'] = point_df['pointScore'].isin(break_point_values)

In [144]:
break_point_total = point_df['isBreakPoint'].sum()
print("Total break points:", break_point_total)
point_df[point_df['isBreakPoint'] == 1].head()


Total break points: 20


,pointNumber,player1Name,player2Name,pointScore,gameScore,setScore,tiebreakScore,side,serverName,returnerName,...,returnFhBh,errorType,returnError,lastShotDirection,lastShotFhBh,lastShotHitBy,lastShotResult,pointWonBy,isExcitingPoint,isBreakPoint
32,7,Elise Wagle,Motka Komac,40-40,0-0,0-0,None,Ad,Motka Komac,Elise Wagle,...,Forehand,Wide Right,None,Crosscourt,Backhand,Motka Komac,Error,Elise Wagle,0.0,True
76,17,Elise Wagle,Motka Komac,30-40,2-0,0-0,None,Ad,Motka Komac,Elise Wagle,...,Forehand,Wide Right,Wide Right,Crosscourt,Forehand,Elise Wagle,Error,Motka Komac,0.0,True
154,38,Elise Wagle,Motka Komac,15-40,4-2,0-0,None,Deuce,Motka Komac,Elise Wagle,...,Forehand,Wide Left,Wide Left,Crosscourt,Forehand,Elise Wagle,Error,Motka Komac,NaN,True
220,51,Elise Wagle,Motka Komac,40-40,5-3,0-0,None,Ad,Motka Komac,Elise Wagle,...,Forehand,None,None,Crosscourt,Forehand,Motka Komac,Winner,Motka Komac,NaN,True
254,56,Elise Wagle,Motka Komac,15-40,5-4,0-0,None,Deuce,Elise Wagle,Motka Komac,...,Backhand,Net,None,Down the Line,Forehand,Elise Wagle,Error,Motka Komac,NaN,True


## Warning: Do not use if atNetPlayer1 is in shot_data
### Add Column: atNetPlayer1, atNetPlayer2

- if below says atNetPlayer1: Uncomment below Code
- replace later with atNetPlayer1 atNetPlayer2

In [146]:
# Check if 'atNetPlayer1' exists in point_df
if 'atNetPlayer1' in shot_data.columns:
    print("Column 'atNetPlayer1' exists in point_df.")
else:
    raise ValueError("Column 'atNetPlayer1' does not exist in point_df.")


if 'atNetPlayer2' in shot_data.columns:
    print("Column 'atNetPlayer2' exists in point_df.")
else:
    raise ValueError("Column 'atNetPlayer2' does not exist in point_df.")

Column 'atNetPlayer1' exists in point_df.
Column 'atNetPlayer2' exists in point_df.


In [147]:
# Add atNetPlayer1 and atNetPlayer2 columns
# point_df['atNetPlayer1'] = 0
# point_df['atNetPlayer2'] = 0

for point_number in shot_data['pointNumber'].unique():
    # Check if firstServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['atNetPlayer1'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'atNetPlayer1'] = 1

    # Check if secondServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['atNetPlayer2'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'atNetPlayer2'] = 1

### Add Column: setNum

In [149]:
# Extract numbers from 'setScore' and calculate the sum plus 1
point_df['setNum'] = point_df['setScore'].apply(lambda x: sum(int(char) for char in x if char.isdigit()) + 1)

In [150]:
point_df.columns

Index(['pointNumber', 'player1Name', 'player2Name', 'pointScore', 'gameScore',
       'setScore', 'tiebreakScore', 'side', 'serverName', 'returnerName',
       'clientTeam', 'opponentTeam', 'Position', 'pointEndPosition',
       'Duration', 'rallyCount', 'rallyCountFreq', 'firstServeIn',
       'secondServeIn', 'serveResult', 'serveInPlacement', 'firstServeZone',
       'secondServeZone', 'isAce', 'serverFarNear', 'firstServeXCoord',
       'firstServeYCoord', 'secondServeXCoord', 'secondServeYCoord',
       'returnDirection', 'returnFhBh', 'errorType', 'returnError',
       'lastShotDirection', 'lastShotFhBh', 'lastShotHitBy', 'lastShotResult',
       'pointWonBy', 'isExcitingPoint', 'isBreakPoint', 'atNetPlayer2',
       'atNetPlayer1', 'setNum'],
      dtype='object')

### Add Column: Game Number, Set Number, Game/Set/Point for each player

In [152]:
point_df[['player1SetScore', 'player2SetScore']] = point_df['setScore'].str.split('-', expand=True)
point_df[['player1GameScore', 'player2GameScore']] = point_df['gameScore'].str.split('-', expand=True)
point_df[['player1PointScore', 'player2PointScore']] = point_df['pointScore'].str.split('-', expand=True)
if not point_df['tiebreakScore'].isnull().all() and not point_df['tiebreakScore'].eq("").all():
    # Perform the operation only when tiebreakScore is not empty
    point_df[['player1TiebreakScore', 'player2TiebreakScore']] = point_df['tiebreakScore'].str.split('-', expand=True)
else:
    # Set player1TiebreakScore and player2TiebreakScore to NaN
    point_df['player1TiebreakScore'] = np.nan
    point_df['player2TiebreakScore'] = np.nan
    
def calculate_game_number(score):
    return int(score.split('-')[0]) + int(score.split('-')[1])+1

# Apply the function to create the 'gameNumber' column
point_df['gameNumber'] = point_df['gameScore'].apply(calculate_game_number)

## Error Check 24: 
#### Output Game Number in order: Should be consecutive increasing. Ex: 1,2,3,4,5,6. End of Set 1. 1,2,3,4,5,6,7,8

In [154]:
game_numbers = point_df['gameNumber'].tolist()

# Initialize variables
seen = set()
prev = None

# Iterate through gameNumber column
for num in game_numbers:
    # If the number is not in seen or it's different from the previous one, print it
    if num not in seen or num != prev:
        print(num, end=', ')
    # If the number is the same as the previous one but not consecutive, print it
    elif num == prev and num not in seen:
        print(num, end=', ')
    # Update seen set and prev variable
    seen.add(num)
    prev = num

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

### Add Column: player1ServeResult

In [156]:
# Add the 'player1ServeResult' column
point_df['player1ServeResult'] = None

# Set player1ServeResult based on conditions
point_df.loc[point_df['serverName'] == point_df['player1Name'], 'player1ServeResult'] = point_df['serveResult']

In [157]:
point_df.loc[point_df['isAce'] == True, 'player1ServeResult'] = 'Ace'


### Add Column: player1ServePlacement

In [159]:
# Add the 'player1ServePlacement' column
point_df['player1ServePlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['serverName'] == point_df['player1Name'], 'player1ServePlacement'] = point_df['side'] + ': ' + point_df['serveInPlacement']

### Add Column: player1ReturnPlacement

In [161]:
# Add the 'player1ReturnPlacement' column
point_df['player1ReturnPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player1Name'], 'player1ReturnPlacement'] = point_df['returnDirection']

### Add Column: player1ReturnFhBh

In [163]:
# Add the 'player1ReturnFhBh' column
point_df['player1ReturnFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player1Name'], 'player1ReturnFhBh'] = point_df['returnFhBh']

### Add Column: player1LastShotPlacement

In [165]:
# Add the 'player1LastShotFhBh' column
point_df['player1LastShotPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player1Name'], 'player1LastShotPlacement'] = point_df['lastShotDirection']

### Add Column: player1LastShotFhBh

In [167]:
# Add the 'player1LastShotFhBh' column
point_df['player1LastShotFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player1Name'], 'player1LastShotFhBh'] = point_df['lastShotFhBh']

### Add Column: player1LastShotResult

In [169]:
# Add the 'player1LastShotResult' column
point_df['player1LastShotResult'] = None

# Set player1LastShotResult based on conditions, excluding 'Ace' and 'Double Fault'
point_df.loc[
    (point_df['lastShotHitBy'] == point_df['player1Name']) & 
    ~point_df['player1ServeResult'].isin(['Ace', 'Double Fault']), 
    'player1LastShotResult'
] = point_df['lastShotResult']


In [170]:
# Add the 'player2ServeResult' column
point_df['player2ServeResult'] = None

# Set player1ServeResult based on conditions
point_df.loc[point_df['serverName'] == point_df['player2Name'], 'player2ServeResult'] = point_df['serveResult']

In [171]:
point_df.loc[point_df['isAce'] == True, 'player1ServeResult'] = 'Ace'


### Add Column: player2ServePlacement

In [173]:
# Add the 'player1ServePlacement' column
point_df['player2ServePlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['serverName'] == point_df['player2Name'], 'player2ServePlacement'] = point_df['side'] + ': ' + point_df['serveInPlacement']

### Add Column: player2ReturnPlacement

In [175]:
# Add the 'player2ReturnPlacement' column
point_df['player2ReturnPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player2Name'], 'player2ReturnPlacement'] = point_df['returnDirection']

### Add Column: player2ReturnFhBh

In [177]:
# Add the 'player1ReturnFhBh' column
point_df['player2ReturnFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player2Name'], 'player2ReturnFhBh'] = point_df['returnFhBh']

### Add Column: player2LastShotPlacement

In [179]:
# Add the 'player1LastShotFhBh' column
point_df['player2LastShotPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player2Name'], 'player2LastShotPlacement'] = point_df['lastShotDirection']

### Add Column: player2LastShotFhBh

In [181]:
# Add the 'player1LastShotFhBh' column
point_df['player2LastShotFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player2Name'], 'player2LastShotFhBh'] = point_df['lastShotFhBh']

### Add Column: player2LastShotResult

In [183]:
# Add the 'player2LastShotResult' column
point_df['player2LastShotResult'] = None

# Set player1LastShotResult based on conditions, excluding 'Ace' and 'Double Fault'
point_df.loc[
    (point_df['lastShotHitBy'] == point_df['player2Name']) &
    ~point_df['player2ServeResult'].isin(['Ace', 'Double Fault']),
    'player2LastShotResult'
] = point_df['lastShotResult']


In [184]:
print(point_df[['setNum', 'gameScore','pointScore','player1ServePlacement','serveInPlacement']].head(10))

    setNum gameScore pointScore player1ServePlacement serveInPlacement
0        1       0-0        0-0                  None             Body
11       1       0-0       0-15                  None             Body
16       1       0-0      15-15                  None             Wide
25       1       0-0      15-30                  None             Body
26       1       0-0      30-30                  None                T
30       1       0-0      40-30                  None                T
32       1       0-0      40-40                  None                T
39       1       1-0        0-0           Deuce: Body             Body
43       1       1-0       15-0              Ad: Wide             Wide
45       1       1-0       30-0           Deuce: Body             Body


In [185]:
print(point_df[['setNum', 'gameScore','pointScore','player1ServePlacement','serveInPlacement']].head(10))

    setNum gameScore pointScore player1ServePlacement serveInPlacement
0        1       0-0        0-0                  None             Body
11       1       0-0       0-15                  None             Body
16       1       0-0      15-15                  None             Wide
25       1       0-0      15-30                  None             Body
26       1       0-0      30-30                  None                T
30       1       0-0      40-30                  None                T
32       1       0-0      40-40                  None                T
39       1       1-0        0-0           Deuce: Body             Body
43       1       1-0       15-0              Ad: Wide             Wide
45       1       1-0       30-0           Deuce: Body             Body


### Add Columns: Depths Count (Short, Deep)

In [187]:
# Add counts for each player on how many short and deep balls they hit in the point (group by pointNumber)
# - don't have to do long since we already know what points end with isErrorLong (and also isErrorNet)


shot_data['deep'] = np.where(shot_data['depth'] == 'Deep', 1, 0)
shot_data['short'] = np.where(shot_data['depth'] == 'Short', 1, 0)


#point_df['deepCountPlayer1'] = shot_data.groupby(['pointNumber', 'player1Name'])['deep'].sum().values
#point_df['shortCountPlayer1'] = shot_data.groupby(['pointNumber', 'player1Name'])['short'].sum().values

#point_df['deepCountPlayer2'] = shot_data.groupby(['pointNumber', 'player2Name'])['deep'].sum().values
#point_df['shortCountPlayer2'] = shot_data.groupby(['pointNumber', 'player2Name'])['short'].sum().values



### attempt 2:

deep_group = shot_data.pivot_table(index='pointNumber', columns='shotHitBy', values='deep', aggfunc='sum')
deep_group.columns = ['deepCountPlayer1', 'deepCountPlayer2']
# change NaN's from double faults to 0?
deep_group.fillna(0, inplace=True)


short_group = shot_data.pivot_table(index='pointNumber', columns='shotHitBy', values='short', aggfunc='sum')
short_group.columns = ['shortCountPlayer1', 'shortCountPlayer2']
# change NaN's from double faults to 0?
short_group.fillna(0, inplace=True)


point_df = pd.merge(point_df, deep_group, how='left', on='pointNumber')
point_df = pd.merge(point_df, short_group, how='left', on='pointNumber')

In [188]:
# CHECK COUNTS:

#print(point_df.head(10))
#print(point_df[['shortCountPlayer1', 'shortCountPlayer2']].head(10))
#print(shot_data.head(10))

## Add Column: Name (For Video)

In [190]:
# Change pointScore to the specified format
point_df['Name'] = point_df.apply(lambda row: f"Set {row['setNum']}: {row['gameScore']}, {row['tiebreakScore']} {row['serverName']} Serving" if pd.notna(row['tiebreakScore']) else f"Set {row['setNum']}: {row['gameScore']}, {row['pointScore']} {row['serverName']} Serving", axis=1)


In [191]:
point_df[['setNum', 'gameScore','pointScore','tiebreakScore','Name']].head(5)

,setNum,gameScore,pointScore,tiebreakScore,Name
0,1,0-0,0-0,None,"Set 1: 0-0, 0-0 Motka Komac Serving"
1,1,0-0,0-15,None,"Set 1: 0-0, 0-15 Motka Komac Serving"
2,1,0-0,15-15,None,"Set 1: 0-0, 15-15 Motka Komac Serving"
3,1,0-0,15-30,None,"Set 1: 0-0, 15-30 Motka Komac Serving"
4,1,0-0,30-30,None,"Set 1: 0-0, 30-30 Motka Komac Serving"


In [192]:
point_df[['setNum', 'gameScore','pointScore','tiebreakScore','Name']].tail(5)

,setNum,gameScore,pointScore,tiebreakScore,Name
125,2,5-3,15-0,None,"Set 2: 5-3, 15-0 Elise Wagle Serving"
126,2,5-3,30-0,None,"Set 2: 5-3, 30-0 Elise Wagle Serving"
127,2,5-3,30-15,None,"Set 2: 5-3, 30-15 Elise Wagle Serving"
128,2,5-3,30-30,None,"Set 2: 5-3, 30-30 Elise Wagle Serving"
129,2,5-3,40-30,None,"Set 2: 5-3, 40-30 Elise Wagle Serving"


## Reorder DataFrame for Output

In [194]:
point_df_copy = point_df.copy()

In [195]:
point_df.shape

(130, 71)

In [196]:
point_df_copy.shape

(130, 71)

In [197]:
point_df.columns

Index(['pointNumber', 'player1Name', 'player2Name', 'pointScore', 'gameScore',
       'setScore', 'tiebreakScore', 'side', 'serverName', 'returnerName',
       'clientTeam', 'opponentTeam', 'Position', 'pointEndPosition',
       'Duration', 'rallyCount', 'rallyCountFreq', 'firstServeIn',
       'secondServeIn', 'serveResult', 'serveInPlacement', 'firstServeZone',
       'secondServeZone', 'isAce', 'serverFarNear', 'firstServeXCoord',
       'firstServeYCoord', 'secondServeXCoord', 'secondServeYCoord',
       'returnDirection', 'returnFhBh', 'errorType', 'returnError',
       'lastShotDirection', 'lastShotFhBh', 'lastShotHitBy', 'lastShotResult',
       'pointWonBy', 'isExcitingPoint', 'isBreakPoint', 'atNetPlayer2',
       'atNetPlayer1', 'setNum', 'player1SetScore', 'player2SetScore',
       'player1GameScore', 'player2GameScore', 'player1PointScore',
       'player2PointScore', 'player1TiebreakScore', 'player2TiebreakScore',
       'gameNumber', 'player1ServeResult', 'player1ServePla

In [198]:
desired_order = [
    'Name', 'pointNumber', 'setNum', 'gameNumber', 'player1Name', 'player2Name',
    'pointScore', 'gameScore', 'setScore', 'tiebreakScore', 'side', 'serverName',
    'returnerName', 'Position', 'pointEndPosition', 'Duration', 'pointWonBy',
    'rallyCount', 'rallyCountFreq', 'firstServeIn', 'secondServeIn',
    'serveResult', 'serveInPlacement', 'firstServeZone', 'secondServeZone',
    # add depth counts !
    'deepCountPlayer1', 'shortCountPlayer1', 'deepCountPlayer2', 'shortCountPlayer2',
    'isAce','serverFarNear', 'firstServeXCoord', 'firstServeYCoord', 'secondServeXCoord', 'secondServeYCoord','returnDirection', 'returnFhBh', 'errorType', 'returnError',
    'lastShotDirection', 'lastShotFhBh', 'lastShotHitBy', 'lastShotResult',
    'isBreakPoint', 'atNetPlayer1', 'atNetPlayer2', 'isExcitingPoint', 'player1SetScore',
    'player2SetScore', 'player1GameScore', 'player2GameScore', 'player1PointScore',
    'player2PointScore', 'player1TiebreakScore', 'player2TiebreakScore',
    'player1ServeResult', 'player1ServePlacement', 'player1ReturnPlacement',
    'player1ReturnFhBh', 'player1LastShotPlacement', 'player1LastShotFhBh',
    'player1LastShotResult', 'player2ServeResult', 'player2ServePlacement',
       'player2ReturnPlacement', 'player2ReturnFhBh',
       'player2LastShotPlacement', 'player2LastShotFhBh',
       'player2LastShotResult', 'clientTeam', 'opponentTeam'
]

# Reorder the columns
point_df = point_df.reindex(columns=desired_order)

## Error Check 25: 
#### Check if the columns and their order are the same

In [200]:
print(point_df.shape)
print(point_df_copy.shape)

if (point_df.shape == point_df_copy.shape):
    print('Check passed.')
else:
    raise ValueError('Error: Not the same!')

(130, 71)
(130, 71)
Check passed.


In [201]:
# Get the set of column names for each DataFrame
point_df_columns = set(point_df.columns)
point_df_copy_columns = set(point_df_copy.columns)

# Find the column names unique to each DataFrame
unique_to_point_df = point_df_columns - point_df_copy_columns
unique_to_point_df_copy = point_df_copy_columns - point_df_columns

# Output the results
if unique_to_point_df:
    print("Columns unique to point_df:", unique_to_point_df)
else:
    print("All columns in point_df are also in point_df_copy")

if unique_to_point_df_copy:
    print("Columns unique to point_df_copy:", unique_to_point_df_copy)
else:
    print("All columns in point_df_copy are also in point_df")

All columns in point_df are also in point_df_copy
All columns in point_df_copy are also in point_df


## Warning: All nan to ""
### Alterating Data

In [203]:
# Change Na to "" Empty String

# Assuming point_df is your DataFrame
point_df.replace([pd.NA, None, pd.NaT, float('nan')], "", inplace=True)

# Double Check before Outputting Point (FOR VISUAL)

In [205]:
print(point_df.head(5))

                                    Name  pointNumber  setNum  gameNumber  \
0    Set 1: 0-0, 0-0 Motka Komac Serving            1       1           1   
1   Set 1: 0-0, 0-15 Motka Komac Serving            2       1           1   
2  Set 1: 0-0, 15-15 Motka Komac Serving            3       1           1   
3  Set 1: 0-0, 15-30 Motka Komac Serving            4       1           1   
4  Set 1: 0-0, 30-30 Motka Komac Serving            5       1           1   

   player1Name  player2Name pointScore gameScore setScore tiebreakScore  ...  \
0  Elise Wagle  Motka Komac        0-0       0-0      0-0                ...   
1  Elise Wagle  Motka Komac       0-15       0-0      0-0                ...   
2  Elise Wagle  Motka Komac      15-15       0-0      0-0                ...   
3  Elise Wagle  Motka Komac      15-30       0-0      0-0                ...   
4  Elise Wagle  Motka Komac      30-30       0-0      0-0                ...   

  player1LastShotResult player2ServeResult player2ServeP

In [206]:
print(point_df.tail(5))

                                      Name  pointNumber  setNum  gameNumber  \
125   Set 2: 5-3, 15-0 Elise Wagle Serving          126       2           9   
126   Set 2: 5-3, 30-0 Elise Wagle Serving          127       2           9   
127  Set 2: 5-3, 30-15 Elise Wagle Serving          128       2           9   
128  Set 2: 5-3, 30-30 Elise Wagle Serving          129       2           9   
129  Set 2: 5-3, 40-30 Elise Wagle Serving          130       2           9   

     player1Name  player2Name pointScore gameScore setScore tiebreakScore  \
125  Elise Wagle  Motka Komac       15-0       5-3      1-0                 
126  Elise Wagle  Motka Komac       30-0       5-3      1-0                 
127  Elise Wagle  Motka Komac      30-15       5-3      1-0                 
128  Elise Wagle  Motka Komac      30-30       5-3      1-0                 
129  Elise Wagle  Motka Komac      40-30       5-3      1-0                 

     ... player1LastShotResult player2ServeResult player2Serve

In [208]:
# Change Value Names for Match Viewer Output

# Assuming point_df is your DataFrame and player1Name is the name from the first row
player1Name = point_df.iloc[0]['player1Name']
player2Name = point_df.iloc[0]['player2Name']

# Replace values in the 'atNetPlayer1' column
point_df['atNetPlayer1'] = point_df['atNetPlayer1'].replace({0: "", 1: player1Name})
point_df['atNetPlayer2'] = point_df['atNetPlayer2'].replace({0: "", 1: player2Name})

# PRINT POINTCSV VISUALS

In [210]:
# Save point_df to CSV file

# Assuming point_df is your DataFrame and player1Name and player2Name are the names from the first row
player1NameNoSpace = point_df.iloc[0]['player1Name'].replace(" ", "")
player2NameNoSpace = point_df.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
point_df.to_csv(f'Point_Visuals_{player1NameNoSpace}_{player2NameNoSpace}.csv', index=False)

In [211]:
def csv_to_json(csv_file_path, json_file_name):
    # Load CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Convert DataFrame to JSON format
    json_data = df.to_json(orient='records')
    
    # Save JSON data to a file in the same directory
    json_file_path = f'{json_file_name}'
    with open(json_file_path, 'w') as json_file:
        json_file.write(json_data)
    
    return json_file_path

# Define the CSV file path
csv_file_path = f"Point_Visuals_{player1NameNoSpace}_{player2NameNoSpace}.csv"

# Define the desired JSON file name (without extension)
json_file_name = f"Point_Visuals_{player1NameNoSpace}_{player2NameNoSpace}.json"

# Convert CSV to JSON and save in the same directory
output_file_path = csv_to_json(csv_file_path, json_file_name)

# Display the path where the JSON file is saved
output_file_path

'Point_Visuals_EliseWagle_MotkaKomac.json'

# EDA

## Shot CSV EDA

In [214]:
# Can input CSV Directly here for statistics functions

# your_file_name = "filename.csv"
# shot_eda = pd.read_csv(your_file_name)

# if directly from notebook
shot_eda = shot_data.copy()

In [215]:
player1Name = shot_eda.iloc[0]['player1Name']

# Filter shot_data based on the conditions
approach_data_player1 = shot_eda[(shot_eda['isApproach'] == 1) & (shot_eda['shotHitBy'] == player1Name)]

# Count the distinct pointNumbers
distinct_point_numbers = approach_data_player1['pointNumber'].nunique()

# Print the result
print(f"Number of Approach Shots hit by {player1Name}: {distinct_point_numbers}" )

# print(approach_data_player1)

Number of Approach Shots hit by Elise Wagle: 5


## Point CSV EDA

In [217]:
# # Can input CSV Directly here for statistics functions
# import pandas as pd
# your_file_name = "Point_Visuals_Anne-ChristineLutkemeyer_SehaYu.csv"
# point_df_eda = pd.read_csv(your_file_name)

# # OR MAKE point_df_eda copy of point_df

point_df_eda = point_df.copy()

In [218]:
print(point_df_eda.columns)

Index(['Name', 'pointNumber', 'setNum', 'gameNumber', 'player1Name',
       'player2Name', 'pointScore', 'gameScore', 'setScore', 'tiebreakScore',
       'side', 'serverName', 'returnerName', 'Position', 'pointEndPosition',
       'Duration', 'pointWonBy', 'rallyCount', 'rallyCountFreq',
       'firstServeIn', 'secondServeIn', 'serveResult', 'serveInPlacement',
       'firstServeZone', 'secondServeZone', 'deepCountPlayer1',
       'shortCountPlayer1', 'deepCountPlayer2', 'shortCountPlayer2', 'isAce',
       'serverFarNear', 'firstServeXCoord', 'firstServeYCoord',
       'secondServeXCoord', 'secondServeYCoord', 'returnDirection',
       'returnFhBh', 'errorType', 'returnError', 'lastShotDirection',
       'lastShotFhBh', 'lastShotHitBy', 'lastShotResult', 'isBreakPoint',
       'atNetPlayer1', 'atNetPlayer2', 'isExcitingPoint', 'player1SetScore',
       'player2SetScore', 'player1GameScore', 'player2GameScore',
       'player1PointScore', 'player2PointScore', 'player1TiebreakScore',
  

## [Depth EDA]

In [392]:
# Filter shots for Player1:
print(f"\n\nShot Results for {first_player1Name} for match overall:")

player1_shots = shot_eda[(shot_eda['shotHitBy'] == player1Name) & (shot_eda['lastShotError'] != 1) & (shot_eda['shotInRally'] != 1)]
#player1_shots = player1_shots[player1_shots['shotInRally'] != 1]
#print(player1_shots.head(10))
num_player1_shots = len(player1_shots)
num_deep_player1_shots = player1_shots['deep'].sum()
num_short_player1_shots = player1_shots['short'].sum()

print(f"Number of Deep Shots (count): {num_deep_player1_shots}" )
print(f"Number of Short Shots (count): {num_short_player1_shots}" )

# average of ALL shots in match for Player1 (including errors/long shots but excluding serves)
print(f"Number of Deep Shots (%): {num_deep_player1_shots / num_player1_shots:.2f}%")
print(f"Number of Short Shots (%): {num_short_player1_shots / num_player1_shots:.2f}%")


# separate into forehand, backhand, slice, and volley (all sepearate from each other):
# [remove slice and volleys from fh and bh count] ??

fh_shots = player1_shots[(player1_shots['shotFhBh'] == 'Forehand')] #& (player1_shots['isSlice'] != 1) & (player1_shots['isVolley'] != 1)]
bh_shots = player1_shots[(player1_shots['shotFhBh'] == 'Backhand')] #& (player1_shots['isSlice'] != 1) & (player1_shots['isVolley'] != 1)]
slice_shots = player1_shots[player1_shots['isSlice'] == 1]
volley_shots = player1_shots[player1_shots['isVolley'] == 1]

fh_slice = player1_shots[(player1_shots['shotFhBh'] == 'Forehand') & (player1_shots['isSlice'] == 1)]
bh_slice = player1_shots[(player1_shots['shotFhBh'] == 'Backhand') & (player1_shots['isSlice'] == 1)]
fh_volley = player1_shots[(player1_shots['shotFhBh'] == 'Forehand') & (player1_shots['isVolley'] == 1)]
bh_volley = player1_shots[(player1_shots['shotFhBh'] == 'Backhand') & (player1_shots['isVolley'] == 1)]

    
print(f"\nTotal number of Forehands (count): {len(fh_shots)}" )
print(f"Forehands Deep (count): {fh_shots['deep'].sum()}" )
print(f"Forehands Short (count): {fh_shots['short'].sum()}" )
print(f"Forehands Deep (%): {( fh_shots['deep'].sum() / len(fh_shots) ):.2f}%")
print(f"Forehands Short (%): {( fh_shots['short'].sum() / len(fh_shots) ):.2f}%")


print(f"\nTotal number of Backhands (count): {len(bh_shots)}" )
print(f"Backhands Deep (count): {bh_shots['deep'].sum()}" )
print(f"Backhands Short (count): {bh_shots['short'].sum()}" )
print(f"Backhands Deep (%): {( bh_shots['deep'].sum() / len(bh_shots) ):.2f}%")
print(f"Backhands Short (%): {( bh_shots['short'].sum() / len(bh_shots) ):.2f}%")


print(f"\nTotal number of Slices (count): {len(slice_shots)}" )
print(f"Slices Deep (count): {slice_shots['deep'].sum()}" )
print(f"Slices Short (count): {slice_shots['short'].sum()}" )
print(f"Slices Deep (%): {( slice_shots['deep'].sum() / len(slice_shots) ):.2f}%")
print(f"Slices Short (%): {( slice_shots['short'].sum() / len(slice_shots) ):.2f}%")

print(f"Slices Deep Forehand(%): {(fh_slice[('deep')].sum() / len(slice_shots) ):.2f}%")
print(f"Slices Short Forehand (%): {(fh_slice[('short')].sum()/ len(slice_shots) ):.2f}%")
print(f"Slices Deep Backhand (%): {(bh_slice[('deep')].sum()/ len(slice_shots) ):.2f}")
print(f"Slices Short Backhand (%): {(bh_slice[('short')].sum()/ len(slice_shots) ):.2f}")


print(f"\nTotal number of Volleys (count): {len(volley_shots)}" )
print(f"Volleys Deep (count): {volley_shots['deep'].sum()}" )
print(f"Volleys Short (count): {volley_shots['short'].sum()}" )
print(f"Volleys Deep (%): {( volley_shots['deep'].sum() / len(volley_shots) ):.2f}%")
print(f"Volleys Short (%): {( volley_shots['short'].sum() / len(volley_shots) ):.2f}%")

print(f"Volleys Deep Forehand (%): {(fh_volley[('deep')].sum()/ len(volley_shots) ):.2f}")
print(f"Volleys Short Forehand (%): {(fh_volley[('short')].sum()/ len(volley_shots) ):.2f}")
print(f"Volleys Deep Backhand (%): {(bh_volley[('deep')].sum()/ len(volley_shots) ):.2f}")
print(f"Volleys Short Backhand (%): {(bh_volley[('short')].sum()/ len(volley_shots) ):.2f}")



Shot Results for Elise Wagle for match overall:
Number of Deep Shots (count): 83
Number of Short Shots (count): 147
Number of Deep Shots (%): 0.35%
Number of Short Shots (%): 0.63%

Total number of Forehands (count): 132
Forehands Deep (count): 43
Forehands Short (count): 84
Forehands Deep (%): 0.33%
Forehands Short (%): 0.64%

Total number of Backhands (count): 103
Backhands Deep (count): 40
Backhands Short (count): 63
Backhands Deep (%): 0.39%
Backhands Short (%): 0.61%

Total number of Slices (count): 16
Slices Deep (count): 6
Slices Short (count): 10
Slices Deep (%): 0.38%
Slices Short (%): 0.62%
Slices Deep Forehand(%): 0.12%
Slices Short Forehand (%): 0.50%
Slices Deep Backhand (%): 0.25
Slices Short Backhand (%): 0.12

Total number of Volleys (count): 4
Volleys Deep (count): 0
Volleys Short (count): 4
Volleys Deep (%): 0.00%
Volleys Short (%): 1.00%
Volleys Deep Forehand (%): 0.00
Volleys Short Forehand (%): 0.50
Volleys Deep Backhand (%): 0.00
Volleys Short Backhand (%): 0.50

In [373]:
# Filter shots for Player1:
print(f"\n\nShot Results for {first_player1Name} per point:")

print(f"Average Deep Shots (%): {(point_df_eda['deepCountPlayer1'].sum()) / len(point_df_eda):.2f}%")
print(f"Average Short Shots (%): {(point_df_eda['shortCountPlayer1'].sum()) / len(point_df_eda):.2f}%")


#print(point_df_eda.columns)
#point_df_eda
#player1_shots = shot_eda[shot_eda['shotHitBy'] == player1Name]

# fix above code: make avg in each point, then avg the avg?



Shot Results for Elise Wagle per point:
Average Deep Shots (%): 0.64%
Average Short Shots (%): 1.13%


In [222]:
serve_in_rows = point_df_eda[(point_df_eda['serveResult'] == '1st Serve In') | (point_df_eda['serveResult'] == '2nd Serve In')]

# filter the rows where rallyCount column is equal to 1
desired_rows = serve_in_rows[serve_in_rows['rallyCount'] == 1]

# print each row
for index, row in desired_rows.iterrows():
    print(row[['pointNumber', 'setNum', 'gameNumber','serveResult','rallyCount' ,'serverName']])


pointNumber              11
setNum                    1
gameNumber                2
serveResult    1st Serve In
rallyCount                1
serverName      Elise Wagle
Name: 10, dtype: object


In [223]:
first_player1Name = point_df_eda['player1Name'].iloc[0]



# Display the results
print(f"\n\nServe Results for {first_player1Name}:")

# Assuming point_df_eda is your DataFrame
total_serves = len(point_df_eda[point_df_eda['serverName'] == first_player1Name])
first_serve_in_count = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['firstServeIn'] == 1)])
first_serve_won_count = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['firstServeIn'] == 1) & (point_df_eda['pointWonBy'] == first_player1Name)])
percentage_first_serve_in = (first_serve_in_count / total_serves) * 100 if total_serves > 0 else 0
percentage_first_serve_won = (first_serve_won_count / first_serve_in_count) * 100 if first_serve_in_count > 0 else 0

second_serve_total_count = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['firstServeIn'] == 0)])
second_serve_in_count = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['firstServeIn'] == 0)& (point_df_eda['secondServeIn'] == 1)])
second_serve_won_count = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['firstServeIn'] == 0)& (point_df_eda['secondServeIn'] == 1) & (point_df_eda['pointWonBy'] == first_player1Name)])
percentage_second_serve_in = (second_serve_in_count / second_serve_total_count) * 100 if second_serve_total_count > 0 else 0
percentage_second_serve_won = (second_serve_won_count / second_serve_in_count) * 100 if second_serve_in_count > 0 else 0



# Display the results
print("\nTotal Serves:", total_serves)
print("First Serve In (Count):", first_serve_in_count)
print("First Serve Won (Count):", first_serve_won_count)
print(f"First Serve In (%): {percentage_first_serve_in:.2f}%")
print(f"First Serve Won (%): {percentage_first_serve_won:.2f}%")

print("Second Serve In (Count):", second_serve_in_count)
print("Second Serve Total (Count):", second_serve_total_count)
print("Second Serve Won (Count):", second_serve_won_count)
print(f"Second Serve In (%): {percentage_second_serve_in:.2f}%")
print(f"Second Serve Won (%): {percentage_second_serve_won:.2f}%")

# Assuming point_df is your DataFrame
count_is_ace = (point_df_eda[point_df_eda['serverName'] == first_player1Name]['isAce']).sum()
count_is_double_fault = ((point_df_eda['serverName'] == first_player1Name) & (point_df_eda['serveResult'] == "Double Fault")).sum()

# Display the results
print("Ace (Count):", count_is_ace)
print("Double Fault (Count):", count_is_double_fault)

# Count of rows where serverName is equal to the first row of player1Name and pointWonBy is equal to the first row of player1Name
total_service_points_won = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['pointWonBy'] == first_player1Name)])
total_service_points_won_percentage = total_service_points_won / total_serves *100

# Display the results
print(f"Points Won on Serve (Count) {total_service_points_won}")

print(f"Points Won on Serve (%): {total_service_points_won_percentage:.2f}%")

# Assuming point_df is your DataFrame
return_points = point_df_eda[(point_df_eda['returnerName'] == first_player1Name)] # CHANGED THIS JERRY, REMOVED RALLY COUNT >= 2

total_return = len(return_points)
returnMade = len(return_points[(return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])
returnError = len(return_points[(return_points['lastShotResult'] == 'Error') & (return_points['rallyCount'] == 2)])
returnWinner = len(return_points[(return_points['lastShotResult'] == 'Winner') & (return_points['rallyCount'] == 2)])
returnMadePercentage = returnMade/total_return

returnWonByPlayer1 = len(return_points[return_points['pointWonBy'] == first_player1Name])
returnWonByPlayer1Percentage = returnWonByPlayer1 / returnMade * 100 if returnMade > 0 else 0

deuceReturnCount = len(return_points[return_points['side'] == 'Deuce'])
adReturnCount = len(return_points[return_points['side'] == 'Ad'])


deuceReturnMade = len(return_points[(return_points['side'] == 'Deuce') & ((return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error')))])
adReturnMade = len(return_points[(return_points['side'] == 'Ad') & ((return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error')))])

deuceReturnMadePercentage = deuceReturnMade/deuceReturnCount
adReturnMadePercentage = adReturnMade/adReturnCount

deuceReturnWonByPlayer1 = len(return_points[(return_points['side'] == 'Deuce') & (return_points['pointWonBy'] == first_player1Name) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])
adReturnWonByPlayer1 = len(return_points[(return_points['side'] == 'Ad') & (return_points['pointWonBy'] == first_player1Name) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])

deuceReturnWonByPlayer1Percentage = deuceReturnWonByPlayer1 / deuceReturnMade * 100 if deuceReturnMade > 0 else 0
adReturnWonByPlayer1Percentage = adReturnWonByPlayer1 / adReturnMade * 100 if adReturnMade > 0 else 0




print(f"\n\nReturn Results for {first_player1Name}:\n")

print("Total Return (Count):", total_return)
print("Return Won (Count):", returnWonByPlayer1)
print("Return Won (%):", returnWonByPlayer1Percentage)

print("\nReturn Made (Count):", returnMade)
print("Return Made (%):", returnMadePercentage)
print("Return Error (Count):", returnError)
print("Return Winner (Count):", returnWinner)

print("\nDeuce Return (Count):", deuceReturnCount)
print("Deuce Return Made (Count):", deuceReturnMade)
print("Deuce Return Made (%):", deuceReturnMadePercentage)
print("Deuce Return Won by Player1 (%):", deuceReturnWonByPlayer1Percentage)
print("Deuce Return Won by Player1 (Count):", deuceReturnWonByPlayer1)


print("\nAd Return (Count):", adReturnCount)
print("Ad Return Made (Count):", adReturnMade)
print("Ad Return Made (%):", adReturnMadePercentage)
print("Ad Return Won by Player1 (Count):", adReturnWonByPlayer1)
print("Ad Return Won by Player1 (%):", adReturnWonByPlayer1Percentage)

# Assuming return_points is your DataFrame
deuce_return_points = return_points[(return_points['side'] == 'Deuce') & (return_points['returnerName'] == first_player1Name) & (return_points['rallyCount'] >= 2)]

# Deuce Return Points Separated by returnFhBh
deuce_forehand_return_points = deuce_return_points[deuce_return_points['returnFhBh'] == 'Forehand']
deuce_backhand_return_points = deuce_return_points[deuce_return_points['returnFhBh'] == 'Backhand']


# Count for Deuce Return Points - Made
count_deuce_forehand_made = len(deuce_forehand_return_points[(deuce_forehand_return_points['rallyCount'] > 2) | ((deuce_forehand_return_points['rallyCount'] == 2) & (deuce_forehand_return_points['lastShotResult'] != 'Error'))])
count_deuce_backhand_made = len(deuce_backhand_return_points[(deuce_backhand_return_points['rallyCount'] > 2) | ((deuce_backhand_return_points['rallyCount'] == 2) & (deuce_backhand_return_points['lastShotResult'] != 'Error'))])

# Count for Deuce Return Points - Error
count_deuce_forehand_error = len(deuce_forehand_return_points[(deuce_forehand_return_points['lastShotResult'] == 'Error') & (deuce_forehand_return_points['rallyCount'] == 2)])
count_deuce_backhand_error = len(deuce_backhand_return_points[(deuce_backhand_return_points['lastShotResult'] == 'Error') & (deuce_backhand_return_points['rallyCount'] == 2)])

# Display the counts
print("\nDeuce Forehand Return Points - Made:", count_deuce_forehand_made)
print("Deuce Forehand Return Points - Error:", count_deuce_forehand_error)

print("Deuce Backhand Return Points - Made:", count_deuce_backhand_made)
print("Deuce Backhand Return Points - Error:", count_deuce_backhand_error)

# Assuming return_points is your DataFrame
ad_return_points = return_points[(return_points['side'] == 'Ad') & (return_points['returnerName'] == first_player1Name) & (return_points['rallyCount'] >= 2)]

# Ad Return Points Separated by returnFhBh
ad_forehand_return_points = ad_return_points[ad_return_points['returnFhBh'] == 'Forehand']
ad_backhand_return_points = ad_return_points[ad_return_points['returnFhBh'] == 'Backhand']

# Count for Ad Return Points - Made
count_ad_forehand_made = len(ad_forehand_return_points[(ad_forehand_return_points['rallyCount'] > 2) | ((ad_forehand_return_points['rallyCount'] == 2) & (ad_forehand_return_points['lastShotResult'] != 'Error'))])
count_ad_backhand_made = len(ad_backhand_return_points[(ad_backhand_return_points['rallyCount'] > 2) | ((ad_backhand_return_points['rallyCount'] == 2) & (ad_backhand_return_points['lastShotResult'] != 'Error'))])

# Count for Ad Return Points - Error
count_ad_forehand_error = len(ad_forehand_return_points[(ad_forehand_return_points['lastShotResult'] == 'Error') & (ad_forehand_return_points['rallyCount'] == 2)])
count_ad_backhand_error = len(ad_backhand_return_points[(ad_backhand_return_points['lastShotResult'] == 'Error') & (ad_backhand_return_points['rallyCount'] == 2)])

# Display the counts
print("\nAd Forehand Return Points - Made:", count_ad_forehand_made)
print("Ad Forehand Return Points - Error:", count_ad_forehand_error)

print("Ad Backhand Return Points - Made:", count_ad_backhand_made)
print("Ad Backhand Return Points - Error:", count_ad_backhand_error)

print(f"\nAt Net Results for {first_player1Name}:\n")


# Total points where atNetPlayer1 = first_player1Name
total_at_net_player1 = len(point_df_eda[point_df_eda['atNetPlayer1'] == first_player1Name])

# Percentage of points where atNetPlayer1 = 1 out of total points
percentage_at_net_player1 = (total_at_net_player1 / len(point_df_eda)) * 100 if len(point_df_eda) > 0 else 0

# Display the total count and percentage of points where atNetPlayer1 = 1
print(f"Total Net Points for {first_player1Name}: {total_at_net_player1}")
print(f"Percentage of Net Points for {first_player1Name}: {percentage_at_net_player1:.2f}%")

# Points where atNetPlayer1 = first_player1Name and pointWonBy = first_player1Name
at_net_player1_and_won_by_player1 = len(point_df_eda[(point_df_eda['atNetPlayer1'] == first_player1Name) & (point_df_eda['pointWonBy'] == first_player1Name)])

# Percentage of points where atNetPlayer1 = first_player1Name and pointWonBy = first_player1Name out of total points where atNetPlayer1 = 1
percentage_at_net_player1_and_won_by_player1 = (at_net_player1_and_won_by_player1 / total_at_net_player1) * 100 if total_at_net_player1 > 0 else 0

# Display the count and percentage of points where atNetPlayer1 = 1 and pointWonBy = first_player1Name
print(f"\nTotal Net Points won by {first_player1Name}: {at_net_player1_and_won_by_player1}")
print(f"Percentage of Net Points won by {first_player1Name}: {percentage_at_net_player1_and_won_by_player1:.2f}%")




Serve Results for Elise Wagle:

Total Serves: 65
First Serve In (Count): 41
First Serve Won (Count): 28
First Serve In (%): 63.08%
First Serve Won (%): 68.29%
Second Serve In (Count): 19
Second Serve Total (Count): 24
Second Serve Won (Count): 13
Second Serve In (%): 79.17%
Second Serve Won (%): 68.42%
Ace (Count): 1
Double Fault (Count): 5
Points Won on Serve (Count) 41
Points Won on Serve (%): 63.08%

Return Results for Elise Wagle:

Total Return (Count): 65
Return Won (Count): 31
Return Won (%): 63.26530612244898

Return Made (Count): 49
Return Made (%): 0.7538461538461538
Return Error (Count): 8
Return Winner (Count): 1

Deuce Return (Count): 33
Deuce Return Made (Count): 24
Deuce Return Made (%): 0.7272727272727273
Deuce Return Won by Player1 (%): 70.83333333333334
Deuce Return Won by Player1 (Count): 17

Ad Return (Count): 32
Ad Return Made (Count): 25
Ad Return Made (%): 0.78125
Ad Return Won by Player1 (Count): 15
Ad Return Won by Player1 (%): 60.0

Deuce Forehand Return Poin

In [224]:
# Caitlin Breakpoint Data

# points_returned = point_df_eda[point_df_eda[]]

# print(f"\nBreakpoint Results for {first_player1Name}:\n")

# # Total points where isBreakPoint = 1
# total_breakpoint = len(point_df_eda[point_df_eda['isBreakPoint'] == 1])

# # Points where isBreakPoint = 1 and pointWonBy = first_player1Name
# breakpoint_and_won_by_player1 = len(point_df_eda[(point_df_eda['isBreakPoint'] == 1) & (point_df_eda['pointWonBy'] == first_player1Name)])

# # Percentage of points where isBreakPoint = 1 and pointWonBy = first_player1Name out of total points where isBreakPoint = 1
# percentage_breakpoint_and_won_by_player1 = (breakpoint_and_won_by_player1 / total_breakpoint) * 100 if total_breakpoint > 0 else 0

# # Display the total count of points where isBreakPoint = 1
# print(f"Total Breakpoints: {total_breakpoint}")

# # Display the count and percentage of points where isBreakPoint = 1 and pointWonBy = first_player1Name
# print(f"Total Breakpoints won by {first_player1Name}: {breakpoint_and_won_by_player1}")
# print(f"Percentage of Breakpoints won by {first_player1Name}: {percentage_breakpoint_and_won_by_player1:.2f}%")

# # Total points where isBreakPoint = 1 and serverName = first_player1Name
# total_breakpoint_serve = len(point_df_eda[(point_df_eda['isBreakPoint'] == 1) & (point_df_eda['serverName'] == first_player1Name)])

# # Points where isBreakPoint = 1, serverName = first_player1Name, and pointWonBy = first_player1Name
# breakpoint_and_won_by_player1_serve = len(point_df_eda[(point_df_eda['isBreakPoint'] == 1) & (point_df_eda['serverName'] == first_player1Name) & (point_df_eda['pointWonBy'] == first_player1Name)])

# # Percentage of points where isBreakPoint = 1, serverName = first_player1Name, and pointWonBy = first_player1Name out of total points where isBreakPoint = 1 and serverName = first_player1Name
# percentage_breakpoint_and_won_by_player1_serve = (breakpoint_and_won_by_player1_serve / total_breakpoint_serve) * 100 if total_breakpoint_serve > 0 else 0

# # Display the total count of points where isBreakPoint = 1 and serverName = first_player1Name
# print(f"\nTotal Breakpoints on Serve for {first_player1Name}: {total_breakpoint_serve}")

# # Display the count and percentage of points where isBreakPoint = 1, serverName = first_player1Name, and pointWonBy = first_player1Name
# print(f"Total Breakpoints won on Serve by {first_player1Name}: {breakpoint_and_won_by_player1_serve}")
# print(f"Percentage of Breakpoints won on Serve by {first_player1Name}: {percentage_breakpoint_and_won_by_player1_serve:.2f}%")

# Total points where isBreakPoint = 1 and returnerName = first_player1Name
total_breakpoint_return = len(point_df_eda[(point_df_eda['isBreakPoint'] == 1) & (point_df_eda['returnerName'] == first_player1Name)])

# Points where isBreakPoint = 1, returnerName = first_player1Name, and pointWonBy = first_player1Name
breakpoint_and_won_by_player1_return = len(point_df_eda[(point_df_eda['isBreakPoint'] == 1) & (point_df_eda['returnerName'] == first_player1Name) & (point_df_eda['pointWonBy'] == first_player1Name)])

# Percentage of points where isBreakPoint = 1, returnerName = first_player1Name, and pointWonBy = first_player1Name out of total points where isBreakPoint = 1 and returnerName = first_player1Name
percentage_breakpoint_and_won_by_player1_return = (breakpoint_and_won_by_player1_return / total_breakpoint_return) * 100 if total_breakpoint_return > 0 else 0

# Display the total count of points where isBreakPoint = 1 and returnerName = first_player1Name
print(f"\nTotal Breakpoints on Return for {first_player1Name}: {total_breakpoint_return}")

# Display the count and percentage of points where isBreakPoint = 1, returnerName = first_player1Name, and pointWonBy = first_player1Name
print(f"Total Breakpoints won on Return by {first_player1Name}: {breakpoint_and_won_by_player1_return}")
print(f"Percentage of Breakpoints won on Return by {first_player1Name}: {percentage_breakpoint_and_won_by_player1_return:.2f}%")

# Jimmy Returning Games Won
# games won/returning games by Jimmy Hou

points_returned = point_df_eda[point_df_eda["returnerName"] == first_player1Name]

# # Return percentage won on first serve
# first_serves_won = points_returned[(points_returned['firstServeIn'] == 1) & (points_returned['pointWonBy'] == first_player1Name)]
# total_first_serves = points_returned[points_returned['firstServeIn'] == 1]
# fs_won_per_player1 = 100 * len(first_serves_won) / len(total_first_serves)
# # print(f"\n{player1} won {fs_won_per_player1:.2f}% of first serves returned.")
# print(f"\nPerecentage of Breakpoinnts won by {first_player1Name} on Return when returning a first serve: {fs_won_per_player1:.2f}%")

# # Return percentage won on second serve
# second_serves_won = points_returned[(points_returned['secondServeIn'] == 1) & (points_returned['pointWonBy'] == first_player1Name)]
# total_second_serves = points_returned[points_returned['secondServeIn'] == 1]
# ss_won_per_player1 = 100 * len(second_serves_won) / len(total_second_serves)
# # print(f"{player1} won {ss_won_per_player1:.2f}% of second serves returned."
# print(f"Perecentage of Breakpoinnts won by {first_player1Name} on Return when returning a second serve: {ss_won_per_player1:.2f}%")



Total Breakpoints on Return for Elise Wagle: 9
Total Breakpoints won on Return by Elise Wagle: 3
Percentage of Breakpoints won on Return by Elise Wagle: 33.33%


In [225]:
point_df_eda.columns

Index(['Name', 'pointNumber', 'setNum', 'gameNumber', 'player1Name',
       'player2Name', 'pointScore', 'gameScore', 'setScore', 'tiebreakScore',
       'side', 'serverName', 'returnerName', 'Position', 'pointEndPosition',
       'Duration', 'pointWonBy', 'rallyCount', 'rallyCountFreq',
       'firstServeIn', 'secondServeIn', 'serveResult', 'serveInPlacement',
       'firstServeZone', 'secondServeZone', 'deepCountPlayer1',
       'shortCountPlayer1', 'deepCountPlayer2', 'shortCountPlayer2', 'isAce',
       'serverFarNear', 'firstServeXCoord', 'firstServeYCoord',
       'secondServeXCoord', 'secondServeYCoord', 'returnDirection',
       'returnFhBh', 'errorType', 'returnError', 'lastShotDirection',
       'lastShotFhBh', 'lastShotHitBy', 'lastShotResult', 'isBreakPoint',
       'atNetPlayer1', 'atNetPlayer2', 'isExcitingPoint', 'player1SetScore',
       'player2SetScore', 'player1GameScore', 'player2GameScore',
       'player1PointScore', 'player2PointScore', 'player1TiebreakScore',
  

In [226]:
# Filter points where serverName is equal to first_player1Name
filtered_points = point_df[point_df['serverName'] == first_player1Name]

# Group the filtered points by player1ServePlacement and count the occurrences
serve_placement_counts = filtered_points.groupby('player1ServePlacement').size()

serve_placements_ad = serve_placement_counts.filter(like='Ad').sum()
serve_placements_deuce = serve_placement_counts.filter(like='Deuce').sum()

# Iterate over filtered_points
for index, point in filtered_points.iterrows():
    serve_placement = point['player1ServePlacement']

    # Check if serve placement is not in serve_placement_counts
    if serve_placement not in serve_placement_counts:
        print(point)


# Initialize dictionaries to store counts and percentages
point_won_counts = {}
point_won_percentages = {}
print(f"Total {len(filtered_points)}")

# Iterate over serve placements
for serve_placement, count in serve_placement_counts.items():
    # Filter points with the specific serve placement
    serve_placement_points = filtered_points[filtered_points['player1ServePlacement'] == serve_placement]
    
    # Count points won by first_player1Name
    point_won_count = serve_placement_points[serve_placement_points['pointWonBy'] == first_player1Name].shape[0]

    # Calculate percentage
    point_won_percentage = (point_won_count / count) * 100 if count > 0 else 0

    # Store counts and percentages
    point_won_counts[serve_placement] = point_won_count
    point_won_percentages[serve_placement] = point_won_percentage
    


    
    
# Filter serve placements for Ad and Deuce [CHANGED: BRIAN NTOES]
serve_placements_ad = serve_placement_counts.filter(like='Ad').sum()
serve_placements_deuce = serve_placement_counts.filter(like='Deuce').sum()    

# Print counts and percentages [CHANGED: BRIAN NOTES]
for serve_placement, count in serve_placement_counts.items():
    print(f"Serve Placement: {serve_placement}")
    print(f"Total Serves: {count}")
    
    if "Deuce" in serve_placement: # [CHANGED: BRIAN NOTES]
        deuce_serve_format_number = f"{count/serve_placements_deuce:.2f}"
        deuce_serve_percent = round(float(deuce_serve_format_number) * 100,2)
        print(f"Serve Frequency: {deuce_serve_percent}% ({count}/{serve_placements_deuce}) Deuce Serves")
    if "Ad" in serve_placement: # [CHANGED: BRIAN NOTES]
        ad_serve_format_number = f"{count/serve_placements_ad:.2f}"
        ad_serve_percent = round(float(ad_serve_format_number) * 100,2)
        print(f"Serve Frequency: {ad_serve_percent}% ({count}/{serve_placements_ad}) Ad Serves")
        
    print(f"Serves Won by {first_player1Name}: {point_won_counts.get(serve_placement, 0)}")
    print(f"Percentage: {point_won_percentages.get(serve_placement, 0):.2f}%\n")

    
# print("This is Ad count: " + str(len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['side'] == 'Ad')])))
# print("This is Deuce count: " + str(len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['side'] == 'Deuce')])))

Total 65
Serve Placement: Ad: Body
Total Serves: 15
Serve Frequency: 45.0% (15/33) Ad Serves
Serves Won by Elise Wagle: 7
Percentage: 46.67%

Serve Placement: Ad: T
Total Serves: 5
Serve Frequency: 15.0% (5/33) Ad Serves
Serves Won by Elise Wagle: 3
Percentage: 60.00%

Serve Placement: Ad: Wide
Total Serves: 13
Serve Frequency: 39.0% (13/33) Ad Serves
Serves Won by Elise Wagle: 11
Percentage: 84.62%

Serve Placement: Deuce: Body
Total Serves: 17
Serve Frequency: 53.0% (17/32) Deuce Serves
Serves Won by Elise Wagle: 11
Percentage: 64.71%

Serve Placement: Deuce: T
Total Serves: 9
Serve Frequency: 28.0% (9/32) Deuce Serves
Serves Won by Elise Wagle: 5
Percentage: 55.56%

Serve Placement: Deuce: Wide
Total Serves: 6
Serve Frequency: 19.0% (6/32) Deuce Serves
Serves Won by Elise Wagle: 4
Percentage: 66.67%



In [227]:
print(f"\nError Data for {first_player1Name}:\n")
# Filter the DataFrame based on specified conditions
total_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) &
                           (point_df['lastShotResult'] == 'Error')]

import numpy as np

# Filter rows without NaN values in relevant columns
forehand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) &
                           (point_df['lastShotResult'] == 'Error') &
                           (point_df['lastShotFhBh'] == 'Forehand') &
                           (~point_df['errorType'].isnull())]  # Ensure 'errorType' column doesn't have NaN
 
backhand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) &
                           (point_df['lastShotResult'] == 'Error') &
                           (point_df['lastShotFhBh'] == 'Backhand') &
                           (~point_df['errorType'].isnull())]  # Ensure 'errorType' column doesn't have NaN

# Count the occurrences of 'Forehand' and 'Backhand' separately
forehand_counts = forehand_errors.shape[0]  # Count rows
backhand_counts = backhand_errors.shape[0]  # Count rows

# Print the total error counts for verification
total_error_counts = forehand_counts + backhand_counts



# Desired output order 
desired_order = ['Net', 'Long', 'Wide Right', 'Wide Left'] 

# # Get value counts of 'errorType' for Forehand errors [CHANGED: BRIAN]
# forehand_error_types = forehand_errors['errorType'].value_counts(dropna=False).loc[desired_order]  # Include NaN values in count
# forehand_error_types_df = pd.DataFrame(forehand_error_types) # change into dataframe to erase object line
# Get value counts of 'errorType' for Backhand errors



######################### CHANGED SECTION ##############################################
# [CHANGED: BRIAN]

forehand_error_types = forehand_errors['errorType'].value_counts(dropna=False)

# Create a Series with desired index containing zeros
zeros_series = pd.Series(0, index=desired_order)

# Combine the original Series with the zeros Series
forehand_error_types_combined = forehand_error_types.combine(zeros_series, max, fill_value=0)

# Reindex the Series to follow the desired order
forehand_error_types_ordered = forehand_error_types_combined.reindex(desired_order, fill_value=0)

# Create the DataFrame
forehand_error_types_df = pd.DataFrame(forehand_error_types_ordered, columns=['Count'])

# Get value counts of 'errorType' for Backhand errors [CHANGED: BRIAN]
# backhand_error_types = backhand_errors['errorType'].value_counts(dropna=False).loc[desired_order]  # Include NaN values in count
# backhand_error_types_df = pd.DataFrame(backhand_error_types) # change into dataframe to erase object line

# Get value counts of 'errorType' for Backhand errors
backhand_error_types = backhand_errors['errorType'].value_counts(dropna=False)

# Combine the original Series with the zeros Series
backhand_error_types_combined = backhand_error_types.combine(zeros_series, max, fill_value=0)

# Reindex the Series to follow the desired order
backhand_error_types_ordered = backhand_error_types_combined.reindex(desired_order, fill_value=0)

# Create the DataFrame
backhand_error_types_df = pd.DataFrame(backhand_error_types_ordered, columns=['Count'])



######################### CHANGED SECTION ##############################################


# Print the counts and error types
print("Count of Total errors:", total_error_counts)
print("Count of Forehand errors:", forehand_counts)
print(f"Forehand Error %: {(forehand_counts/total_error_counts)*100:.2f}%")
print("Count of Backhand errors:", backhand_counts)
print(f"Backhand Error %: {(backhand_counts/total_error_counts)*100:.2f}%")
print("\nForehand errors:\n", forehand_error_types_df)


# # Group by both 'lastShotDirection' and 'errorType', and then count occurrences
forehand_error_counts = forehand_errors.groupby(['player1LastShotPlacement', 'errorType']).size().unstack(fill_value=0)  # Fill NaN with 0
forehand_error_counts_ordered = forehand_error_counts.reindex(columns = desired_order) # [CHANGED: BRIAN]

print("\nValue counts of 'errorType' for Forehand errors with different directions:\n", forehand_error_counts_ordered)


print("\nBackhand errors:\n", backhand_error_types_df)


# Group by both 'lastShotDirection' and 'errorType', and then count occurrences
backhand_error_counts = backhand_errors.groupby(['player1LastShotPlacement', 'errorType']).size().unstack(fill_value=0) # Fill NaN with 0
backhand_error_counts_ordered = backhand_error_counts.reindex(columns = desired_order) # [CHANGED: BRIAN]

print("\nValue counts of 'errorType' for Backhand errors with different directions:\n", backhand_error_counts_ordered)


Error Data for Elise Wagle:

Count of Total errors: 37
Count of Forehand errors: 24
Forehand Error %: 64.86%
Count of Backhand errors: 13
Backhand Error %: 35.14%

Forehand errors:
             Count
Net             8
Long            5
Wide Right      7
Wide Left       4

Value counts of 'errorType' for Forehand errors with different directions:
 errorType                 Net  Long  Wide Right  Wide Left
player1LastShotPlacement                                  
Crosscourt                  2     2           3          3
Down the Line               6     3           4          1

Backhand errors:
             Count
Net             5
Long            3
Wide Right      4
Wide Left       1

Value counts of 'errorType' for Backhand errors with different directions:
 errorType                 Net  Long  Wide Right  Wide Left
player1LastShotPlacement                                  
Crosscourt                  1     2           4          0
Down the Line               4     1           0    

# CONDENSED REPORT (NEEDS UPDATE)

In [229]:
# pip install reportlab

In [230]:
### from reportlab.lib import colors
# from reportlab.lib.pagesizes import letter
# from reportlab.platypus import SimpleDocTemplate, Table, TableStyle

# # Your raw output data
# raw_data = [
#     {"Serve Placement": "Ad: Body", "Total Serves": 3, "Serves Won by Tian Fang Ran": 3, "Percentage": "100.00%"},
#     {"Serve Placement": "Ad: T", "Total Serves": 11, "Serves Won by Tian Fang Ran": 8, "Percentage": "72.73%"},
#     {"Serve Placement": "Ad: Wide", "Total Serves": 19, "Serves Won by Tian Fang Ran": 10, "Percentage": "52.63%"},
#     {"Serve Placement": "Deuce: Body", "Total Serves": 5, "Serves Won by Tian Fang Ran": 2, "Percentage": "40.00%"},
#     {"Serve Placement": "Deuce: T", "Total Serves": 19, "Serves Won by Tian Fang Ran": 10, "Percentage": "52.63%"},
#     {"Serve Placement": "Deuce: Wide", "Total Serves": 14, "Serves Won by Tian Fang Ran": 7, "Percentage": "50.00%"}
# ]

# # Convert raw data to list of lists for table
# table_data = [["Serve Placement", "Total Serves", "Serves Won by Tian Fang Ran", "Percentage"]]
# for row in raw_data:
#     table_data.append([row["Serve Placement"], row["Total Serves"], row["Serves Won by Tian Fang Ran"], row["Percentage"]])

# # Create PDF
# pdf_filename = "tennis_stats.pdf"
# doc = SimpleDocTemplate(pdf_filename, pagesize=letter)
# table = Table(table_data)

# # Add style to table
# style = TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.gray),
#                     ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
#                     ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
#                     ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
#                     ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
#                     ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
#                     ('GRID', (0, 0), (-1, -1), 1, colors.black)])
# table.setStyle(style)

# # Add table to PDF
# doc.build([table])
